In [4]:
import requests
cookies = {
    'forever_region_id': '1',
    'selected_region_subdomain': 'www',
    'session_main_town_region_id': '1',
    'session_region_id': '1',
    'save_search_animation_done': 'rawr',
}
headers = {
    'Pragma': 'no-cache',
    'Origin': 'https://www.cian.ru',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,es;q=0.6',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'DNT': '1',
}
data = {"_subdomain":"www","_path":"/cat.php?contract%5B0%5D=4&deal_type=sale&engine_version=2&is_first_floor=1&offer_type=offices&office_type%5B0%5D=2&region=1"}
response = requests.post('https://www.cian.ru/cian-api/mobile-site/v2/search-offers/', headers=headers, cookies=cookies, data=data)
response.json()

{'status': 'badRequest',
 'data': {'message': 'Request body is not valid JSON object'}}

In [2]:
import json
data = response.json()["data"]
offers = data["offersSerialized"]
offers

KeyError: 'offersSerialized'

In [4]:
# This code parses date/times, so please
#
#     pip install python-dateutil
#
# To use this code, make sure you
#
#     import json
#
# and then, to convert JSON from a string, do
#
#     result = cian_from_dict(json.loads(json_string))

from dataclasses import dataclass
from typing import Optional, Any, List, Dict, TypeVar, Callable, Type, cast
from enum import Enum
from datetime import datetime
from uuid import UUID
import dateutil.parser


T = TypeVar("T")
EnumT = TypeVar("EnumT", bound=Enum)


def from_str(x: Any) -> str:
    assert isinstance(x, str)
    return x


def from_none(x: Any) -> Any:
    assert x is None
    return x


def from_union(fs, x):
    for f in fs:
        try:
            return f(x)
        except:
            pass
    assert False


def from_list(f: Callable[[Any], T], x: Any) -> List[T]:
    assert isinstance(x, list)
    return [f(y) for y in x]


def from_int(x: Any) -> int:
    assert isinstance(x, int) and not isinstance(x, bool)
    return x


def to_enum(c: Type[EnumT], x: Any) -> EnumT:
    assert isinstance(x, c)
    return x.value


def from_bool(x: Any) -> bool:
    assert isinstance(x, bool)
    return x


def to_class(c: Type[T], x: Any) -> dict:
    assert isinstance(x, c)
    return cast(Any, x).to_dict()


def from_dict(f: Callable[[Any], T], x: Any) -> Dict[str, T]:
    assert isinstance(x, dict)
    return { k: f(v) for (k, v) in x.items() }


def from_float(x: Any) -> float:
    assert isinstance(x, (float, int)) and not isinstance(x, bool)
    return float(x)


def to_float(x: Any) -> float:
    assert isinstance(x, float)
    return x


def is_type(t: Type[T], x: Any) -> T:
    assert isinstance(x, t)
    return x


def from_datetime(x: Any) -> datetime:
    return dateutil.parser.parse(x)


@dataclass
class Breadcrumb:
    url: Optional[str]
    title: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'Breadcrumb':
        assert isinstance(obj, dict)
        url = from_union([from_str, from_none], obj.get("url"))
        title = from_union([from_str, from_none], obj.get("title"))
        return Breadcrumb(url, title)

    def to_dict(self) -> dict:
        result: dict = {}
        result["url"] = from_union([from_str, from_none], self.url)
        result["title"] = from_union([from_str, from_none], self.title)
        return result


class ContractType(Enum):
    TERMS = "terms"


@dataclass
class Contract:
    type: Optional[ContractType]
    value: Optional[List[int]]

    @staticmethod
    def from_dict(obj: Any) -> 'Contract':
        assert isinstance(obj, dict)
        type = from_union([ContractType, from_none], obj.get("type"))
        value = from_union([lambda x: from_list(from_int, x), from_none], obj.get("value"))
        return Contract(type, value)

    def to_dict(self) -> dict:
        result: dict = {}
        result["type"] = from_union([lambda x: to_enum(ContractType, x), from_none], self.type)
        result["value"] = from_union([lambda x: from_list(from_int, x), from_none], self.value)
        return result


class EngineVersionType(Enum):
    TERM = "term"


@dataclass
class EngineVersion:
    type: Optional[EngineVersionType]
    value: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'EngineVersion':
        assert isinstance(obj, dict)
        type = from_union([EngineVersionType, from_none], obj.get("type"))
        value = from_union([from_int, from_none], obj.get("value"))
        return EngineVersion(type, value)

    def to_dict(self) -> dict:
        result: dict = {}
        result["type"] = from_union([lambda x: to_enum(EngineVersionType, x), from_none], self.type)
        result["value"] = from_union([from_int, from_none], self.value)
        return result


@dataclass
class IsFirstFloor:
    type: Optional[EngineVersionType]
    value: Optional[bool]

    @staticmethod
    def from_dict(obj: Any) -> 'IsFirstFloor':
        assert isinstance(obj, dict)
        type = from_union([EngineVersionType, from_none], obj.get("type"))
        value = from_union([from_bool, from_none], obj.get("value"))
        return IsFirstFloor(type, value)

    def to_dict(self) -> dict:
        result: dict = {}
        result["type"] = from_union([lambda x: to_enum(EngineVersionType, x), from_none], self.type)
        result["value"] = from_union([from_bool, from_none], self.value)
        return result


class TypeEnum(Enum):
    COMMERCIALSALE = "commercialsale"


@dataclass
class JSONQuery:
    type: Optional[TypeEnum]
    region: Optional[Contract]
    office_type: Optional[Contract]
    engine_version: Optional[EngineVersion]
    contract: Optional[Contract]
    is_first_floor: Optional[IsFirstFloor]

    @staticmethod
    def from_dict(obj: Any) -> 'JSONQuery':
        assert isinstance(obj, dict)
        type = from_union([TypeEnum, from_none], obj.get("_type"))
        region = from_union([Contract.from_dict, from_none], obj.get("region"))
        office_type = from_union([Contract.from_dict, from_none], obj.get("office_type"))
        engine_version = from_union([EngineVersion.from_dict, from_none], obj.get("engine_version"))
        contract = from_union([Contract.from_dict, from_none], obj.get("contract"))
        is_first_floor = from_union([IsFirstFloor.from_dict, from_none], obj.get("is_first_floor"))
        return JSONQuery(type, region, office_type, engine_version, contract, is_first_floor)

    def to_dict(self) -> dict:
        result: dict = {}
        result["_type"] = from_union([lambda x: to_enum(TypeEnum, x), from_none], self.type)
        result["region"] = from_union([lambda x: to_class(Contract, x), from_none], self.region)
        result["office_type"] = from_union([lambda x: to_class(Contract, x), from_none], self.office_type)
        result["engine_version"] = from_union([lambda x: to_class(EngineVersion, x), from_none], self.engine_version)
        result["contract"] = from_union([lambda x: to_class(Contract, x), from_none], self.contract)
        result["is_first_floor"] = from_union([lambda x: to_class(IsFirstFloor, x), from_none], self.is_first_floor)
        return result


class DealTypeEnum(Enum):
    SALE = "sale"


class Currency(Enum):
    RUR = "rur"


@dataclass
class TPrices:
    rur: Optional[int]
    usd: Optional[int]
    eur: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'TPrices':
        assert isinstance(obj, dict)
        rur = from_union([from_int, from_none], obj.get("rur"))
        usd = from_union([from_int, from_none], obj.get("usd"))
        eur = from_union([from_int, from_none], obj.get("eur"))
        return TPrices(rur, usd, eur)

    def to_dict(self) -> dict:
        result: dict = {}
        result["rur"] = from_union([from_int, from_none], self.rur)
        result["usd"] = from_union([from_int, from_none], self.usd)
        result["eur"] = from_union([from_int, from_none], self.eur)
        return result


class PriceTypeEnum(Enum):
    ALL = "all"


class VatType(Enum):
    INCLUDED = "included"
    USN = "usn"


@dataclass
class BargainTerms:
    price_rur: Optional[int]
    contract_type: Optional[DealTypeEnum]
    included_options: Optional[List[Any]]
    vat_price: Optional[int]
    sale_type: Optional[str]
    price: Optional[int]
    vat_type: Optional[VatType]
    price_usd: Optional[int]
    vat_included: Optional[bool]
    currency: Optional[Currency]
    deposit: Optional[int]
    deposit_prices: Optional[TPrices]
    vat_prices: Optional[TPrices]
    price_eur: Optional[int]
    price_type: Optional[PriceTypeEnum]

    @staticmethod
    def from_dict(obj: Any) -> 'BargainTerms':
        assert isinstance(obj, dict)
        price_rur = from_union([from_int, from_none], obj.get("priceRur"))
        contract_type = from_union([DealTypeEnum, from_none], obj.get("contractType"))
        included_options = from_union([lambda x: from_list(lambda x: x, x), from_none], obj.get("includedOptions"))
        vat_price = from_union([from_int, from_none], obj.get("vatPrice"))
        sale_type = from_union([from_str, from_none], obj.get("saleType"))
        price = from_union([from_int, from_none], obj.get("price"))
        vat_type = from_union([VatType, from_none], obj.get("vatType"))
        price_usd = from_union([from_int, from_none], obj.get("priceUsd"))
        vat_included = from_union([from_bool, from_none], obj.get("vatIncluded"))
        currency = from_union([Currency, from_none], obj.get("currency"))
        deposit = from_union([from_int, from_none], obj.get("deposit"))
        deposit_prices = from_union([TPrices.from_dict, from_none], obj.get("depositPrices"))
        vat_prices = from_union([TPrices.from_dict, from_none], obj.get("vatPrices"))
        price_eur = from_union([from_int, from_none], obj.get("priceEur"))
        price_type = from_union([PriceTypeEnum, from_none], obj.get("priceType"))
        return BargainTerms(price_rur, contract_type, included_options, vat_price, sale_type, price, vat_type, price_usd, vat_included, currency, deposit, deposit_prices, vat_prices, price_eur, price_type)

    def to_dict(self) -> dict:
        result: dict = {}
        result["priceRur"] = from_union([from_int, from_none], self.price_rur)
        result["contractType"] = from_union([lambda x: to_enum(DealTypeEnum, x), from_none], self.contract_type)
        result["includedOptions"] = from_union([lambda x: from_list(lambda x: x, x), from_none], self.included_options)
        result["vatPrice"] = from_union([from_int, from_none], self.vat_price)
        result["saleType"] = from_union([from_str, from_none], self.sale_type)
        result["price"] = from_union([from_int, from_none], self.price)
        result["vatType"] = from_union([lambda x: to_enum(VatType, x), from_none], self.vat_type)
        result["priceUsd"] = from_union([from_int, from_none], self.price_usd)
        result["vatIncluded"] = from_union([from_bool, from_none], self.vat_included)
        result["currency"] = from_union([lambda x: to_enum(Currency, x), from_none], self.currency)
        result["deposit"] = from_union([from_int, from_none], self.deposit)
        result["depositPrices"] = from_union([lambda x: to_class(TPrices, x), from_none], self.deposit_prices)
        result["vatPrices"] = from_union([lambda x: to_class(TPrices, x), from_none], self.vat_prices)
        result["priceEur"] = from_union([from_int, from_none], self.price_eur)
        result["priceType"] = from_union([lambda x: to_enum(PriceTypeEnum, x), from_none], self.price_type)
        return result


class OpeningHoursType(Enum):
    SPECIFIC = "specific"


@dataclass
class OpeningHours:
    type: Optional[OpeningHoursType]

    @staticmethod
    def from_dict(obj: Any) -> 'OpeningHours':
        assert isinstance(obj, dict)
        type = from_union([OpeningHoursType, from_none], obj.get("type"))
        return OpeningHours(type)

    def to_dict(self) -> dict:
        result: dict = {}
        result["type"] = from_union([lambda x: to_enum(OpeningHoursType, x), from_none], self.type)
        return result


class StatusType(Enum):
    OPERATIONAL = "operational"
    UNDER_CONSTRUCTION = "underConstruction"


@dataclass
class Building:
    floors_count: Optional[int]
    cranage_types: Optional[List[Any]]
    total_area: Optional[str]
    type: Optional[str]
    lift_types: Optional[List[Any]]
    name: Optional[str]
    heating_type: Optional[str]
    status_type: Optional[StatusType]
    ceiling_height: Optional[str]
    build_year: Optional[int]
    opening_hours: Optional[OpeningHours]
    infrastructure: Optional[Dict[str, bool]]
    class_type: Optional[str]
    access_type: Optional[str]
    material_type: Optional[str]
    extinguishing_system_type: Optional[str]
    conditioning_type: Optional[str]
    house_material_type: Optional[str]
    ventilation_type: Optional[str]
    house_line_type: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'Building':
        assert isinstance(obj, dict)
        floors_count = from_union([from_int, from_none], obj.get("floorsCount"))
        cranage_types = from_union([lambda x: from_list(lambda x: x, x), from_none], obj.get("cranageTypes"))
        total_area = from_union([from_str, from_none], obj.get("totalArea"))
        type = from_union([from_str, from_none], obj.get("type"))
        lift_types = from_union([lambda x: from_list(lambda x: x, x), from_none], obj.get("liftTypes"))
        name = from_union([from_str, from_none], obj.get("name"))
        heating_type = from_union([from_str, from_none], obj.get("heatingType"))
        status_type = from_union([StatusType, from_none], obj.get("statusType"))
        ceiling_height = from_union([from_str, from_none], obj.get("ceilingHeight"))
        build_year = from_union([from_int, from_none], obj.get("buildYear"))
        opening_hours = from_union([OpeningHours.from_dict, from_none], obj.get("openingHours"))
        infrastructure = from_union([lambda x: from_dict(from_bool, x), from_none], obj.get("infrastructure"))
        class_type = from_union([from_str, from_none], obj.get("classType"))
        access_type = from_union([from_str, from_none], obj.get("accessType"))
        material_type = from_union([from_str, from_none], obj.get("materialType"))
        extinguishing_system_type = from_union([from_str, from_none], obj.get("extinguishingSystemType"))
        conditioning_type = from_union([from_str, from_none], obj.get("conditioningType"))
        house_material_type = from_union([from_str, from_none], obj.get("houseMaterialType"))
        ventilation_type = from_union([from_str, from_none], obj.get("ventilationType"))
        house_line_type = from_union([from_str, from_none], obj.get("houseLineType"))
        return Building(floors_count, cranage_types, total_area, type, lift_types, name, heating_type, status_type, ceiling_height, build_year, opening_hours, infrastructure, class_type, access_type, material_type, extinguishing_system_type, conditioning_type, house_material_type, ventilation_type, house_line_type)

    def to_dict(self) -> dict:
        result: dict = {}
        result["floorsCount"] = from_union([from_int, from_none], self.floors_count)
        result["cranageTypes"] = from_union([lambda x: from_list(lambda x: x, x), from_none], self.cranage_types)
        result["totalArea"] = from_union([from_str, from_none], self.total_area)
        result["type"] = from_union([from_str, from_none], self.type)
        result["liftTypes"] = from_union([lambda x: from_list(lambda x: x, x), from_none], self.lift_types)
        result["name"] = from_union([from_str, from_none], self.name)
        result["heatingType"] = from_union([from_str, from_none], self.heating_type)
        result["statusType"] = from_union([lambda x: to_enum(StatusType, x), from_none], self.status_type)
        result["ceilingHeight"] = from_union([from_str, from_none], self.ceiling_height)
        result["buildYear"] = from_union([from_int, from_none], self.build_year)
        result["openingHours"] = from_union([lambda x: to_class(OpeningHours, x), from_none], self.opening_hours)
        result["infrastructure"] = from_union([lambda x: from_dict(from_bool, x), from_none], self.infrastructure)
        result["classType"] = from_union([from_str, from_none], self.class_type)
        result["accessType"] = from_union([from_str, from_none], self.access_type)
        result["materialType"] = from_union([from_str, from_none], self.material_type)
        result["extinguishingSystemType"] = from_union([from_str, from_none], self.extinguishing_system_type)
        result["conditioningType"] = from_union([from_str, from_none], self.conditioning_type)
        result["houseMaterialType"] = from_union([from_str, from_none], self.house_material_type)
        result["ventilationType"] = from_union([from_str, from_none], self.ventilation_type)
        result["houseLineType"] = from_union([from_str, from_none], self.house_line_type)
        return result


@dataclass
class BusinessShoppingCenter:
    from_representative: Optional[bool]
    id: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'BusinessShoppingCenter':
        assert isinstance(obj, dict)
        from_representative = from_union([from_bool, from_none], obj.get("fromRepresentative"))
        id = from_union([from_int, from_none], obj.get("id"))
        return BusinessShoppingCenter(from_representative, id)

    def to_dict(self) -> dict:
        result: dict = {}
        result["fromRepresentative"] = from_union([from_bool, from_none], self.from_representative)
        result["id"] = from_union([from_int, from_none], self.id)
        return result


class Category(Enum):
    SHOPPING_AREA_SALE = "shoppingAreaSale"


class ConditionType(Enum):
    COSMETIC_REPAIRS_REQUIRED = "cosmeticRepairsRequired"
    DESIGN = "design"
    FINISHING = "finishing"
    TYPICAL = "typical"


class GeoType(Enum):
    GEO = "geo"


class ValueType(Enum):
    DISTRICT = "district"
    HOUSE = "house"
    STREET = "street"
    UNDERGROUND = "underground"


@dataclass
class ValueElement:
    type: Optional[ValueType]
    id: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'ValueElement':
        assert isinstance(obj, dict)
        type = from_union([ValueType, from_none], obj.get("type"))
        id = from_union([from_int, from_none], obj.get("id"))
        return ValueElement(type, id)

    def to_dict(self) -> dict:
        result: dict = {}
        result["type"] = from_union([lambda x: to_enum(ValueType, x), from_none], self.type)
        result["id"] = from_union([from_int, from_none], self.id)
        return result


@dataclass
class DemandJSONQueryGeo:
    type: Optional[GeoType]
    value: Optional[List[ValueElement]]

    @staticmethod
    def from_dict(obj: Any) -> 'DemandJSONQueryGeo':
        assert isinstance(obj, dict)
        type = from_union([GeoType, from_none], obj.get("type"))
        value = from_union([lambda x: from_list(ValueElement.from_dict, x), from_none], obj.get("value"))
        return DemandJSONQueryGeo(type, value)

    def to_dict(self) -> dict:
        result: dict = {}
        result["type"] = from_union([lambda x: to_enum(GeoType, x), from_none], self.type)
        result["value"] = from_union([lambda x: from_list(lambda x: to_class(ValueElement, x), x), from_none], self.value)
        return result


class PriceType(Enum):
    RANGE = "range"


@dataclass
class PriceValue:
    gte: Optional[int]
    lte: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'PriceValue':
        assert isinstance(obj, dict)
        gte = from_union([from_int, from_none], obj.get("gte"))
        lte = from_union([from_int, from_none], obj.get("lte"))
        return PriceValue(gte, lte)

    def to_dict(self) -> dict:
        result: dict = {}
        result["gte"] = from_union([from_int, from_none], self.gte)
        result["lte"] = from_union([from_int, from_none], self.lte)
        return result


@dataclass
class Price:
    type: Optional[PriceType]
    value: Optional[PriceValue]

    @staticmethod
    def from_dict(obj: Any) -> 'Price':
        assert isinstance(obj, dict)
        type = from_union([PriceType, from_none], obj.get("type"))
        value = from_union([PriceValue.from_dict, from_none], obj.get("value"))
        return Price(type, value)

    def to_dict(self) -> dict:
        result: dict = {}
        result["type"] = from_union([lambda x: to_enum(PriceType, x), from_none], self.type)
        result["value"] = from_union([lambda x: to_class(PriceValue, x), from_none], self.value)
        return result


@dataclass
class DemandJSONQuery:
    type: Optional[TypeEnum]
    office_type: Optional[Contract]
    price: Optional[Price]
    total_area: Optional[Price]
    engine_version: Optional[EngineVersion]
    currency: Optional[EngineVersion]
    geo: Optional[DemandJSONQueryGeo]
    region: Optional[Contract]

    @staticmethod
    def from_dict(obj: Any) -> 'DemandJSONQuery':
        assert isinstance(obj, dict)
        type = from_union([TypeEnum, from_none], obj.get("_type"))
        office_type = from_union([Contract.from_dict, from_none], obj.get("office_type"))
        price = from_union([Price.from_dict, from_none], obj.get("price"))
        total_area = from_union([Price.from_dict, from_none], obj.get("total_area"))
        engine_version = from_union([EngineVersion.from_dict, from_none], obj.get("engine_version"))
        currency = from_union([EngineVersion.from_dict, from_none], obj.get("currency"))
        geo = from_union([DemandJSONQueryGeo.from_dict, from_none], obj.get("geo"))
        region = from_union([Contract.from_dict, from_none], obj.get("region"))
        return DemandJSONQuery(type, office_type, price, total_area, engine_version, currency, geo, region)

    def to_dict(self) -> dict:
        result: dict = {}
        result["_type"] = from_union([lambda x: to_enum(TypeEnum, x), from_none], self.type)
        result["office_type"] = from_union([lambda x: to_class(Contract, x), from_none], self.office_type)
        result["price"] = from_union([lambda x: to_class(Price, x), from_none], self.price)
        result["total_area"] = from_union([lambda x: to_class(Price, x), from_none], self.total_area)
        result["engine_version"] = from_union([lambda x: to_class(EngineVersion, x), from_none], self.engine_version)
        result["currency"] = from_union([lambda x: to_class(EngineVersion, x), from_none], self.currency)
        result["geo"] = from_union([lambda x: to_class(DemandJSONQueryGeo, x), from_none], self.geo)
        result["region"] = from_union([lambda x: to_class(Contract, x), from_none], self.region)
        return result


@dataclass
class Electricity:
    possible_to_connect: None
    power: Optional[int]
    location_type: None

    @staticmethod
    def from_dict(obj: Any) -> 'Electricity':
        assert isinstance(obj, dict)
        possible_to_connect = from_none(obj.get("possibleToConnect"))
        power = from_union([from_int, from_none], obj.get("power"))
        location_type = from_none(obj.get("locationType"))
        return Electricity(possible_to_connect, power, location_type)

    def to_dict(self) -> dict:
        result: dict = {}
        result["possibleToConnect"] = from_none(self.possible_to_connect)
        result["power"] = from_union([from_int, from_none], self.power)
        result["locationType"] = from_none(self.location_type)
        return result


@dataclass
class Flags:
    is_archived: Optional[bool]

    @staticmethod
    def from_dict(obj: Any) -> 'Flags':
        assert isinstance(obj, dict)
        is_archived = from_union([from_bool, from_none], obj.get("isArchived"))
        return Flags(is_archived)

    def to_dict(self) -> dict:
        result: dict = {}
        result["isArchived"] = from_union([from_bool, from_none], self.is_archived)
        return result


class AddressType(Enum):
    HOUSE = "house"
    LOCATION = "location"
    STREET = "street"


@dataclass
class Address:
    name: Optional[str]
    id: Optional[int]
    location_type_id: Optional[int]
    full_name: Optional[str]
    type: Optional[AddressType]
    short_name: Optional[str]
    is_forming_address: Optional[bool]

    @staticmethod
    def from_dict(obj: Any) -> 'Address':
        assert isinstance(obj, dict)
        name = from_union([from_str, from_none], obj.get("name"))
        id = from_union([from_int, from_none], obj.get("id"))
        location_type_id = from_union([from_int, from_none], obj.get("locationTypeId"))
        full_name = from_union([from_str, from_none], obj.get("fullName"))
        type = from_union([AddressType, from_none], obj.get("type"))
        short_name = from_union([from_str, from_none], obj.get("shortName"))
        is_forming_address = from_union([from_bool, from_none], obj.get("isFormingAddress"))
        return Address(name, id, location_type_id, full_name, type, short_name, is_forming_address)

    def to_dict(self) -> dict:
        result: dict = {}
        result["name"] = from_union([from_str, from_none], self.name)
        result["id"] = from_union([from_int, from_none], self.id)
        result["locationTypeId"] = from_union([from_int, from_none], self.location_type_id)
        result["fullName"] = from_union([from_str, from_none], self.full_name)
        result["type"] = from_union([lambda x: to_enum(AddressType, x), from_none], self.type)
        result["shortName"] = from_union([from_str, from_none], self.short_name)
        result["isFormingAddress"] = from_union([from_bool, from_none], self.is_forming_address)
        return result


@dataclass
class Coordinates:
    lat: Optional[float]
    lng: Optional[float]

    @staticmethod
    def from_dict(obj: Any) -> 'Coordinates':
        assert isinstance(obj, dict)
        lat = from_union([from_float, from_none], obj.get("lat"))
        lng = from_union([from_float, from_none], obj.get("lng"))
        return Coordinates(lat, lng)

    def to_dict(self) -> dict:
        result: dict = {}
        result["lat"] = from_union([to_float, from_none], self.lat)
        result["lng"] = from_union([to_float, from_none], self.lng)
        return result


class DistrictType(Enum):
    OKRUG = "okrug"
    POSELENIE = "poselenie"
    RAION = "raion"


@dataclass
class District:
    location_id: Optional[int]
    parent_id: Optional[int]
    type: Optional[DistrictType]
    id: Optional[int]
    name: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'District':
        assert isinstance(obj, dict)
        location_id = from_union([from_int, from_none], obj.get("locationId"))
        parent_id = from_union([from_int, from_none], obj.get("parentId"))
        type = from_union([DistrictType, from_none], obj.get("type"))
        id = from_union([from_int, from_none], obj.get("id"))
        name = from_union([from_str, from_none], obj.get("name"))
        return District(location_id, parent_id, type, id, name)

    def to_dict(self) -> dict:
        result: dict = {}
        result["locationId"] = from_union([from_int, from_none], self.location_id)
        result["parentId"] = from_union([from_int, from_none], self.parent_id)
        result["type"] = from_union([lambda x: to_enum(DistrictType, x), from_none], self.type)
        result["id"] = from_union([from_int, from_none], self.id)
        result["name"] = from_union([from_str, from_none], self.name)
        return result


@dataclass
class Highway:
    distance: Optional[str]
    id: Optional[int]
    is_default: Optional[bool]
    name: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'Highway':
        assert isinstance(obj, dict)
        distance = from_union([from_str, from_none], obj.get("distance"))
        id = from_union([from_int, from_none], obj.get("id"))
        is_default = from_union([from_bool, from_none], obj.get("isDefault"))
        name = from_union([from_str, from_none], obj.get("name"))
        return Highway(distance, id, is_default, name)

    def to_dict(self) -> dict:
        result: dict = {}
        result["distance"] = from_union([from_str, from_none], self.distance)
        result["id"] = from_union([from_int, from_none], self.id)
        result["isDefault"] = from_union([from_bool, from_none], self.is_default)
        result["name"] = from_union([from_str, from_none], self.name)
        return result


@dataclass
class GaGeo:
    city_id: Optional[int]
    mo_id: Optional[int]
    obl_id: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'GaGeo':
        assert isinstance(obj, dict)
        city_id = from_union([from_int, from_none], obj.get("cityId"))
        mo_id = from_union([from_int, from_none], obj.get("moId"))
        obl_id = from_union([from_int, from_none], obj.get("oblId"))
        return GaGeo(city_id, mo_id, obl_id)

    def to_dict(self) -> dict:
        result: dict = {}
        result["cityId"] = from_union([from_int, from_none], self.city_id)
        result["moId"] = from_union([from_int, from_none], self.mo_id)
        result["oblId"] = from_union([from_int, from_none], self.obl_id)
        return result


@dataclass
class JkHouse:
    name: Optional[str]
    id: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'JkHouse':
        assert isinstance(obj, dict)
        name = from_union([from_str, from_none], obj.get("name"))
        id = from_union([from_int, from_none], obj.get("id"))
        return JkHouse(name, id)

    def to_dict(self) -> dict:
        result: dict = {}
        result["name"] = from_union([from_str, from_none], self.name)
        result["id"] = from_union([from_int, from_none], self.id)
        return result


@dataclass
class Jk:
    display_name: Optional[str]
    name: Optional[str]
    house: Optional[JkHouse]
    full_url: Optional[str]
    web_site_url: Optional[str]
    ga_geo: Optional[GaGeo]
    web_site_url_utm: Optional[str]
    id: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'Jk':
        assert isinstance(obj, dict)
        display_name = from_union([from_str, from_none], obj.get("displayName"))
        name = from_union([from_str, from_none], obj.get("name"))
        house = from_union([JkHouse.from_dict, from_none], obj.get("house"))
        full_url = from_union([from_str, from_none], obj.get("fullUrl"))
        web_site_url = from_union([from_str, from_none], obj.get("webSiteUrl"))
        ga_geo = from_union([GaGeo.from_dict, from_none], obj.get("gaGeo"))
        web_site_url_utm = from_union([from_str, from_none], obj.get("webSiteUrlUtm"))
        id = from_union([from_int, from_none], obj.get("id"))
        return Jk(display_name, name, house, full_url, web_site_url, ga_geo, web_site_url_utm, id)

    def to_dict(self) -> dict:
        result: dict = {}
        result["displayName"] = from_union([from_str, from_none], self.display_name)
        result["name"] = from_union([from_str, from_none], self.name)
        result["house"] = from_union([lambda x: to_class(JkHouse, x), from_none], self.house)
        result["fullUrl"] = from_union([from_str, from_none], self.full_url)
        result["webSiteUrl"] = from_union([from_str, from_none], self.web_site_url)
        result["gaGeo"] = from_union([lambda x: to_class(GaGeo, x), from_none], self.ga_geo)
        result["webSiteUrlUtm"] = from_union([from_str, from_none], self.web_site_url_utm)
        result["id"] = from_union([from_int, from_none], self.id)
        return result


@dataclass
class LocationPath:
    country_id: Optional[int]
    child_to_parent: Optional[List[int]]

    @staticmethod
    def from_dict(obj: Any) -> 'LocationPath':
        assert isinstance(obj, dict)
        country_id = from_union([from_int, from_none], obj.get("countryId"))
        child_to_parent = from_union([lambda x: from_list(from_int, x), from_none], obj.get("childToParent"))
        return LocationPath(country_id, child_to_parent)

    def to_dict(self) -> dict:
        result: dict = {}
        result["countryId"] = from_union([from_int, from_none], self.country_id)
        result["childToParent"] = from_union([lambda x: from_list(from_int, x), from_none], self.child_to_parent)
        return result


class TransportType(Enum):
    TRANSPORT = "transport"
    WALK = "walk"


@dataclass
class Underground:
    transport_type: Optional[TransportType]
    name: Optional[str]
    under_construction: Optional[bool]
    line_color: Optional[str]
    time: Optional[int]
    line_id: Optional[int]
    id: Optional[int]
    is_default: Optional[bool]

    @staticmethod
    def from_dict(obj: Any) -> 'Underground':
        assert isinstance(obj, dict)
        transport_type = from_union([TransportType, from_none], obj.get("transportType"))
        name = from_union([from_str, from_none], obj.get("name"))
        under_construction = from_union([from_bool, from_none], obj.get("underConstruction"))
        line_color = from_union([from_str, from_none], obj.get("lineColor"))
        time = from_union([from_int, from_none], obj.get("time"))
        line_id = from_union([from_int, from_none], obj.get("lineId"))
        id = from_union([from_int, from_none], obj.get("id"))
        is_default = from_union([from_bool, from_none], obj.get("isDefault"))
        return Underground(transport_type, name, under_construction, line_color, time, line_id, id, is_default)

    def to_dict(self) -> dict:
        result: dict = {}
        result["transportType"] = from_union([lambda x: to_enum(TransportType, x), from_none], self.transport_type)
        result["name"] = from_union([from_str, from_none], self.name)
        result["underConstruction"] = from_union([from_bool, from_none], self.under_construction)
        result["lineColor"] = from_union([from_str, from_none], self.line_color)
        result["time"] = from_union([from_int, from_none], self.time)
        result["lineId"] = from_union([from_int, from_none], self.line_id)
        result["id"] = from_union([from_int, from_none], self.id)
        result["isDefault"] = from_union([from_bool, from_none], self.is_default)
        return result


@dataclass
class OffersSerializedGeo:
    highways: Optional[List[Highway]]
    country_id: Optional[int]
    undergrounds: Optional[List[Underground]]
    coordinates: Optional[Coordinates]
    districts: Optional[List[District]]
    user_input: Optional[str]
    address: Optional[List[Address]]
    location_path: Optional[LocationPath]
    jk: Optional[Jk]

    @staticmethod
    def from_dict(obj: Any) -> 'OffersSerializedGeo':
        assert isinstance(obj, dict)
        highways = from_union([lambda x: from_list(Highway.from_dict, x), from_none], obj.get("highways"))
        country_id = from_union([from_int, from_none], obj.get("countryId"))
        undergrounds = from_union([lambda x: from_list(Underground.from_dict, x), from_none], obj.get("undergrounds"))
        coordinates = from_union([Coordinates.from_dict, from_none], obj.get("coordinates"))
        districts = from_union([lambda x: from_list(District.from_dict, x), from_none], obj.get("districts"))
        user_input = from_union([from_str, from_none], obj.get("userInput"))
        address = from_union([lambda x: from_list(Address.from_dict, x), from_none], obj.get("address"))
        location_path = from_union([LocationPath.from_dict, from_none], obj.get("locationPath"))
        jk = from_union([Jk.from_dict, from_none], obj.get("jk"))
        return OffersSerializedGeo(highways, country_id, undergrounds, coordinates, districts, user_input, address, location_path, jk)

    def to_dict(self) -> dict:
        result: dict = {}
        result["highways"] = from_union([lambda x: from_list(lambda x: to_class(Highway, x), x), from_none], self.highways)
        result["countryId"] = from_union([from_int, from_none], self.country_id)
        result["undergrounds"] = from_union([lambda x: from_list(lambda x: to_class(Underground, x), x), from_none], self.undergrounds)
        result["coordinates"] = from_union([lambda x: to_class(Coordinates, x), from_none], self.coordinates)
        result["districts"] = from_union([lambda x: from_list(lambda x: to_class(District, x), x), from_none], self.districts)
        result["userInput"] = from_union([from_str, from_none], self.user_input)
        result["address"] = from_union([lambda x: from_list(lambda x: to_class(Address, x), x), from_none], self.address)
        result["locationPath"] = from_union([lambda x: to_class(LocationPath, x), from_none], self.location_path)
        result["jk"] = from_union([lambda x: to_class(Jk, x), from_none], self.jk)
        return result


class AreaUnitType(Enum):
    HECTARE = "hectare"


class LandType(Enum):
    OWNED = "owned"
    RENT = "rent"


@dataclass
class Land:
    status: None
    type: Optional[LandType]
    area_unit_type: Optional[AreaUnitType]
    possible_to_change_status: None
    area: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'Land':
        assert isinstance(obj, dict)
        status = from_none(obj.get("status"))
        type = from_union([LandType, from_none], obj.get("type"))
        area_unit_type = from_union([AreaUnitType, from_none], obj.get("areaUnitType"))
        possible_to_change_status = from_none(obj.get("possibleToChangeStatus"))
        area = from_union([from_none, from_str], obj.get("area"))
        return Land(status, type, area_unit_type, possible_to_change_status, area)

    def to_dict(self) -> dict:
        result: dict = {}
        result["status"] = from_none(self.status)
        result["type"] = from_union([lambda x: to_enum(LandType, x), from_none], self.type)
        result["areaUnitType"] = from_union([lambda x: to_enum(AreaUnitType, x), from_none], self.area_unit_type)
        result["possibleToChangeStatus"] = from_none(self.possible_to_change_status)
        result["area"] = from_union([from_none, from_str], self.area)
        return result


@dataclass
class FinishDate:
    quarter: Optional[int]
    year: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'FinishDate':
        assert isinstance(obj, dict)
        quarter = from_union([from_int, from_none], obj.get("quarter"))
        year = from_union([from_int, from_none], obj.get("year"))
        return FinishDate(quarter, year)

    def to_dict(self) -> dict:
        result: dict = {}
        result["quarter"] = from_union([from_int, from_none], self.quarter)
        result["year"] = from_union([from_int, from_none], self.year)
        return result


@dataclass
class NewbuildingHouse:
    is_reliable: Optional[bool]
    is_finished: Optional[bool]
    finish_date: Optional[FinishDate]
    id: Optional[int]
    name: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'NewbuildingHouse':
        assert isinstance(obj, dict)
        is_reliable = from_union([from_bool, from_none], obj.get("isReliable"))
        is_finished = from_union([from_bool, from_none], obj.get("isFinished"))
        finish_date = from_union([FinishDate.from_dict, from_none], obj.get("finishDate"))
        id = from_union([from_int, from_none], obj.get("id"))
        name = from_union([from_str, from_none], obj.get("name"))
        return NewbuildingHouse(is_reliable, is_finished, finish_date, id, name)

    def to_dict(self) -> dict:
        result: dict = {}
        result["isReliable"] = from_union([from_bool, from_none], self.is_reliable)
        result["isFinished"] = from_union([from_bool, from_none], self.is_finished)
        result["finishDate"] = from_union([lambda x: to_class(FinishDate, x), from_none], self.finish_date)
        result["id"] = from_union([from_int, from_none], self.id)
        result["name"] = from_union([from_str, from_none], self.name)
        return result


class DeadlineInfo(Enum):
    СДАН = "Сдан"
    СДАЧА_В_1_КВ_2019 = "Сдача в 1 кв. 2019"
    СДАЧА_В_1_КВ_2020_ЕСТЬ_СДАННЫЕ = "Сдача в 1 кв. 2020, есть сданные"
    СДАЧА_В_20192020_ЕСТЬ_СДАННЫЕ = "Сдача в 2019—2020, есть сданные"


@dataclass
class NewbuildingFeatures:
    images_count: Optional[int]
    reviews_count: Optional[int]
    deadline_info: Optional[DeadlineInfo]
    videos_count: Optional[int]
    first_image_url: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'NewbuildingFeatures':
        assert isinstance(obj, dict)
        images_count = from_union([from_int, from_none], obj.get("imagesCount"))
        reviews_count = from_union([from_int, from_none], obj.get("reviewsCount"))
        deadline_info = from_union([DeadlineInfo, from_none], obj.get("deadlineInfo"))
        videos_count = from_union([from_int, from_none], obj.get("videosCount"))
        first_image_url = from_union([from_str, from_none], obj.get("firstImageUrl"))
        return NewbuildingFeatures(images_count, reviews_count, deadline_info, videos_count, first_image_url)

    def to_dict(self) -> dict:
        result: dict = {}
        result["imagesCount"] = from_union([from_int, from_none], self.images_count)
        result["reviewsCount"] = from_union([from_int, from_none], self.reviews_count)
        result["deadlineInfo"] = from_union([lambda x: to_enum(DeadlineInfo, x), from_none], self.deadline_info)
        result["videosCount"] = from_union([from_int, from_none], self.videos_count)
        result["firstImageUrl"] = from_union([from_str, from_none], self.first_image_url)
        return result


@dataclass
class Newbuilding:
    show_jk_reliable_flag: Optional[bool]
    is_from_developer: Optional[bool]
    name: Optional[str]
    is_from_builder: Optional[bool]
    house: Optional[NewbuildingHouse]
    is_from_seller: Optional[bool]
    is_reliable: Optional[bool]
    newbuilding_features: Optional[NewbuildingFeatures]
    id: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'Newbuilding':
        assert isinstance(obj, dict)
        show_jk_reliable_flag = from_union([from_bool, from_none], obj.get("showJkReliableFlag"))
        is_from_developer = from_union([from_bool, from_none], obj.get("isFromDeveloper"))
        name = from_union([from_str, from_none], obj.get("name"))
        is_from_builder = from_union([from_bool, from_none], obj.get("isFromBuilder"))
        house = from_union([NewbuildingHouse.from_dict, from_none], obj.get("house"))
        is_from_seller = from_union([from_bool, from_none], obj.get("isFromSeller"))
        is_reliable = from_union([from_bool, from_none], obj.get("isReliable"))
        newbuilding_features = from_union([NewbuildingFeatures.from_dict, from_none], obj.get("newbuildingFeatures"))
        id = from_union([from_none, lambda x: int(from_str(x))], obj.get("id"))
        return Newbuilding(show_jk_reliable_flag, is_from_developer, name, is_from_builder, house, is_from_seller, is_reliable, newbuilding_features, id)

    def to_dict(self) -> dict:
        result: dict = {}
        result["showJkReliableFlag"] = from_union([from_bool, from_none], self.show_jk_reliable_flag)
        result["isFromDeveloper"] = from_union([from_bool, from_none], self.is_from_developer)
        result["name"] = from_union([from_str, from_none], self.name)
        result["isFromBuilder"] = from_union([from_bool, from_none], self.is_from_builder)
        result["house"] = from_union([lambda x: to_class(NewbuildingHouse, x), from_none], self.house)
        result["isFromSeller"] = from_union([from_bool, from_none], self.is_from_seller)
        result["isReliable"] = from_union([from_bool, from_none], self.is_reliable)
        result["newbuildingFeatures"] = from_union([lambda x: to_class(NewbuildingFeatures, x), from_none], self.newbuilding_features)
        result["id"] = from_union([lambda x: from_none((lambda x: is_type(type(None), x))(x)), lambda x: from_str((lambda x: str((lambda x: is_type(int, x))(x)))(x))], self.id)
        return result


@dataclass
class Notes:
    realtor: None
    offer: None

    @staticmethod
    def from_dict(obj: Any) -> 'Notes':
        assert isinstance(obj, dict)
        realtor = from_none(obj.get("realtor"))
        offer = from_none(obj.get("offer"))
        return Notes(realtor, offer)

    def to_dict(self) -> dict:
        result: dict = {}
        result["realtor"] = from_none(self.realtor)
        result["offer"] = from_none(self.offer)
        return result


class OfferType(Enum):
    COMMERCIAL = "commercial"


@dataclass
class Phone:
    number: Optional[str]
    country_code: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'Phone':
        assert isinstance(obj, dict)
        number = from_union([from_str, from_none], obj.get("number"))
        country_code = from_union([from_str, from_none], obj.get("countryCode"))
        return Phone(number, country_code)

    def to_dict(self) -> dict:
        result: dict = {}
        result["number"] = from_union([from_str, from_none], self.number)
        result["countryCode"] = from_union([from_str, from_none], self.country_code)
        return result


@dataclass
class Photo:
    thumbnail2_url: Optional[str]
    coordinates: None
    id: Optional[int]
    source: None
    rotate_degree: None
    thumbnail_url: Optional[str]
    full_url: Optional[str]
    is_default: Optional[bool]
    mini_url: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'Photo':
        assert isinstance(obj, dict)
        thumbnail2_url = from_union([from_str, from_none], obj.get("thumbnail2Url"))
        coordinates = from_none(obj.get("coordinates"))
        id = from_union([from_int, from_none], obj.get("id"))
        source = from_none(obj.get("source"))
        rotate_degree = from_none(obj.get("rotateDegree"))
        thumbnail_url = from_union([from_str, from_none], obj.get("thumbnailUrl"))
        full_url = from_union([from_str, from_none], obj.get("fullUrl"))
        is_default = from_union([from_bool, from_none], obj.get("isDefault"))
        mini_url = from_union([from_str, from_none], obj.get("miniUrl"))
        return Photo(thumbnail2_url, coordinates, id, source, rotate_degree, thumbnail_url, full_url, is_default, mini_url)

    def to_dict(self) -> dict:
        result: dict = {}
        result["thumbnail2Url"] = from_union([from_str, from_none], self.thumbnail2_url)
        result["coordinates"] = from_none(self.coordinates)
        result["id"] = from_union([from_int, from_none], self.id)
        result["source"] = from_none(self.source)
        result["rotateDegree"] = from_none(self.rotate_degree)
        result["thumbnailUrl"] = from_union([from_str, from_none], self.thumbnail_url)
        result["fullUrl"] = from_union([from_str, from_none], self.full_url)
        result["isDefault"] = from_union([from_bool, from_none], self.is_default)
        result["miniUrl"] = from_union([from_str, from_none], self.mini_url)
        return result


class PlacementType(Enum):
    STREET_RETAIL = "streetRetail"


class PlatformType(Enum):
    IOS = "ios"
    UPLOAD = "upload"
    WEB_SITE = "webSite"


@dataclass
class Platform:
    version: Optional[str]
    type: Optional[PlatformType]

    @staticmethod
    def from_dict(obj: Any) -> 'Platform':
        assert isinstance(obj, dict)
        version = from_union([from_none, from_str], obj.get("version"))
        type = from_union([PlatformType, from_none], obj.get("type"))
        return Platform(version, type)

    def to_dict(self) -> dict:
        result: dict = {}
        result["version"] = from_union([from_none, from_str], self.version)
        result["type"] = from_union([lambda x: to_enum(PlatformType, x), from_none], self.type)
        return result


@dataclass
class PriceData:
    payment_period: None
    currency: Optional[Currency]
    price: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'PriceData':
        assert isinstance(obj, dict)
        payment_period = from_none(obj.get("paymentPeriod"))
        currency = from_union([Currency, from_none], obj.get("currency"))
        price = from_union([from_int, from_none], obj.get("price"))
        return PriceData(payment_period, currency, price)

    def to_dict(self) -> dict:
        result: dict = {}
        result["paymentPeriod"] = from_none(self.payment_period)
        result["currency"] = from_union([lambda x: to_enum(Currency, x), from_none], self.currency)
        result["price"] = from_union([from_int, from_none], self.price)
        return result


@dataclass
class PriceChange:
    change_time: Optional[datetime]
    price_data: Optional[PriceData]

    @staticmethod
    def from_dict(obj: Any) -> 'PriceChange':
        assert isinstance(obj, dict)
        change_time = from_union([from_datetime, from_none], obj.get("changeTime"))
        price_data = from_union([PriceData.from_dict, from_none], obj.get("priceData"))
        return PriceChange(change_time, price_data)

    def to_dict(self) -> dict:
        result: dict = {}
        result["changeTime"] = from_union([lambda x: x.isoformat(), from_none], self.change_time)
        result["priceData"] = from_union([lambda x: to_class(PriceData, x), from_none], self.price_data)
        return result


class Service(Enum):
    AUCTION = "auction"
    HIGHLIGHT = "highlight"
    PREMIUM = "premium"
    TOP3 = "top3"


class TermType(Enum):
    DAILY_LIMITED = "dailyLimited"
    DAILY_TERMLESS = "dailyTermless"
    PERIODICAL = "periodical"


@dataclass
class Term:
    tariff_identificator: None
    is_compulsory: None
    days: Optional[int]
    services: Optional[List[Service]]
    type: Optional[TermType]
    dynamic_price: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'Term':
        assert isinstance(obj, dict)
        tariff_identificator = from_none(obj.get("tariffIdentificator"))
        is_compulsory = from_none(obj.get("isCompulsory"))
        days = from_union([from_int, from_none], obj.get("days"))
        services = from_union([lambda x: from_list(Service, x), from_none], obj.get("services"))
        type = from_union([TermType, from_none], obj.get("type"))
        dynamic_price = from_union([from_none, from_str], obj.get("dynamicPrice"))
        return Term(tariff_identificator, is_compulsory, days, services, type, dynamic_price)

    def to_dict(self) -> dict:
        result: dict = {}
        result["tariffIdentificator"] = from_none(self.tariff_identificator)
        result["isCompulsory"] = from_none(self.is_compulsory)
        result["days"] = from_union([from_int, from_none], self.days)
        result["services"] = from_union([lambda x: from_list(lambda x: to_enum(Service, x), x), from_none], self.services)
        result["type"] = from_union([lambda x: to_enum(TermType, x), from_none], self.type)
        result["dynamicPrice"] = from_union([from_none, from_str], self.dynamic_price)
        return result


@dataclass
class PublishTerms:
    autoprolong: Optional[bool]
    terms: Optional[List[Term]]

    @staticmethod
    def from_dict(obj: Any) -> 'PublishTerms':
        assert isinstance(obj, dict)
        autoprolong = from_union([from_bool, from_none], obj.get("autoprolong"))
        terms = from_union([lambda x: from_list(Term.from_dict, x), from_none], obj.get("terms"))
        return PublishTerms(autoprolong, terms)

    def to_dict(self) -> dict:
        result: dict = {}
        result["autoprolong"] = from_union([from_bool, from_none], self.autoprolong)
        result["terms"] = from_union([lambda x: from_list(lambda x: to_class(Term, x), x), from_none], self.terms)
        return result


@dataclass
class SpecialtyElement:
    rus_name: Optional[str]
    eng_name: Optional[str]
    id: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'SpecialtyElement':
        assert isinstance(obj, dict)
        rus_name = from_union([from_str, from_none], obj.get("rusName"))
        eng_name = from_union([from_str, from_none], obj.get("engName"))
        id = from_union([from_int, from_none], obj.get("id"))
        return SpecialtyElement(rus_name, eng_name, id)

    def to_dict(self) -> dict:
        result: dict = {}
        result["rusName"] = from_union([from_str, from_none], self.rus_name)
        result["engName"] = from_union([from_str, from_none], self.eng_name)
        result["id"] = from_union([from_int, from_none], self.id)
        return result


@dataclass
class OffersSerializedSpecialty:
    additional_types: Optional[List[Any]]
    specialties: Optional[List[SpecialtyElement]]
    types: Optional[List[str]]

    @staticmethod
    def from_dict(obj: Any) -> 'OffersSerializedSpecialty':
        assert isinstance(obj, dict)
        additional_types = from_union([lambda x: from_list(lambda x: x, x), from_none], obj.get("additionalTypes"))
        specialties = from_union([from_none, lambda x: from_list(SpecialtyElement.from_dict, x)], obj.get("specialties"))
        types = from_union([lambda x: from_list(from_str, x), from_none], obj.get("types"))
        return OffersSerializedSpecialty(additional_types, specialties, types)

    def to_dict(self) -> dict:
        result: dict = {}
        result["additionalTypes"] = from_union([lambda x: from_list(lambda x: x, x), from_none], self.additional_types)
        result["specialties"] = from_union([from_none, lambda x: from_list(lambda x: to_class(SpecialtyElement, x), x)], self.specialties)
        result["types"] = from_union([lambda x: from_list(from_str, x), from_none], self.types)
        return result


@dataclass
class Statistic:
    total: Optional[int]
    daily: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'Statistic':
        assert isinstance(obj, dict)
        total = from_union([from_int, from_none], obj.get("total"))
        daily = from_union([from_int, from_none], obj.get("daily"))
        return Statistic(total, daily)

    def to_dict(self) -> dict:
        result: dict = {}
        result["total"] = from_union([from_int, from_none], self.total)
        result["daily"] = from_union([from_int, from_none], self.daily)
        return result


class Status(Enum):
    PUBLISHED = "published"


class AgentAccountType(Enum):
    AGENCY = "agency"
    MANAGEMENT_COMPANY = "managementCompany"
    SPECIALIST = "specialist"


@dataclass
class AgentAvailability:
    available: Optional[bool]
    user_id: Optional[int]
    available_from: Optional[datetime]
    title: Optional[str]
    available_to: Optional[datetime]
    message: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'AgentAvailability':
        assert isinstance(obj, dict)
        available = from_union([from_bool, from_none], obj.get("available"))
        user_id = from_union([from_int, from_none], obj.get("userId"))
        available_from = from_union([from_datetime, from_none], obj.get("availableFrom"))
        title = from_union([from_str, from_none], obj.get("title"))
        available_to = from_union([from_datetime, from_none], obj.get("availableTo"))
        message = from_union([from_str, from_none], obj.get("message"))
        return AgentAvailability(available, user_id, available_from, title, available_to, message)

    def to_dict(self) -> dict:
        result: dict = {}
        result["available"] = from_union([from_bool, from_none], self.available)
        result["userId"] = from_union([from_int, from_none], self.user_id)
        result["availableFrom"] = from_union([lambda x: x.isoformat(), from_none], self.available_from)
        result["title"] = from_union([from_str, from_none], self.title)
        result["availableTo"] = from_union([lambda x: x.isoformat(), from_none], self.available_to)
        result["message"] = from_union([from_str, from_none], self.message)
        return result


@dataclass
class AgentReviews:
    entity_rate: Optional[int]
    approved_reviews_count: Optional[int]
    total_count: Optional[int]

    @staticmethod
    def from_dict(obj: Any) -> 'AgentReviews':
        assert isinstance(obj, dict)
        entity_rate = from_union([from_int, from_none], obj.get("entityRate"))
        approved_reviews_count = from_union([from_int, from_none], obj.get("approvedReviewsCount"))
        total_count = from_union([from_int, from_none], obj.get("totalCount"))
        return AgentReviews(entity_rate, approved_reviews_count, total_count)

    def to_dict(self) -> dict:
        result: dict = {}
        result["entityRate"] = from_union([from_int, from_none], self.entity_rate)
        result["approvedReviewsCount"] = from_union([from_int, from_none], self.approved_reviews_count)
        result["totalCount"] = from_union([from_int, from_none], self.total_count)
        return result


class CianProfileStatus(Enum):
    APPROVED = "approved"
    HIDE = "hide"


class UserTrustLevel(Enum):
    INVOLVED = "involved"
    NOT_INVOLVED = "notInvolved"


class UserType(Enum):
    REALTOR_BASED = "realtor_based"
    REALTOR_NOT_COMMERCE = "realtor_not_commerce"
    REALTOR_PROF = "realtor_prof"


@dataclass
class User:
    phone_numbers: Optional[List[Phone]]
    user_trust_level: Optional[UserTrustLevel]
    agency_name: Optional[str]
    agent_availability: Optional[AgentAvailability]
    agency_name_v2: Optional[str]
    cian_user_id: Optional[int]
    is_chats_enabled: Optional[bool]
    company_name: Optional[str]
    experience: Optional[str]
    is_sub_agent: Optional[bool]
    user_type: Optional[UserType]
    agent_account_type: Optional[AgentAccountType]
    is_agent: Optional[bool]
    agent_avatar_url: Optional[str]
    is_hidden: Optional[bool]
    cian_profile_status: Optional[CianProfileStatus]
    agent_reviews: Optional[AgentReviews]

    @staticmethod
    def from_dict(obj: Any) -> 'User':
        assert isinstance(obj, dict)
        phone_numbers = from_union([lambda x: from_list(Phone.from_dict, x), from_none], obj.get("phoneNumbers"))
        user_trust_level = from_union([UserTrustLevel, from_none], obj.get("userTrustLevel"))
        agency_name = from_union([from_str, from_none], obj.get("agencyName"))
        agent_availability = from_union([AgentAvailability.from_dict, from_none], obj.get("agentAvailability"))
        agency_name_v2 = from_union([from_str, from_none], obj.get("agencyNameV2"))
        cian_user_id = from_union([from_int, from_none], obj.get("cianUserId"))
        is_chats_enabled = from_union([from_bool, from_none], obj.get("isChatsEnabled"))
        company_name = from_union([from_str, from_none], obj.get("companyName"))
        experience = from_union([from_str, from_none], obj.get("experience"))
        is_sub_agent = from_union([from_bool, from_none], obj.get("isSubAgent"))
        user_type = from_union([UserType, from_none], obj.get("userType"))
        agent_account_type = from_union([AgentAccountType, from_none], obj.get("agentAccountType"))
        is_agent = from_union([from_bool, from_none], obj.get("isAgent"))
        agent_avatar_url = from_union([from_str, from_none], obj.get("agentAvatarUrl"))
        is_hidden = from_union([from_bool, from_none], obj.get("isHidden"))
        cian_profile_status = from_union([CianProfileStatus, from_none], obj.get("cianProfileStatus"))
        agent_reviews = from_union([AgentReviews.from_dict, from_none], obj.get("agentReviews"))
        return User(phone_numbers, user_trust_level, agency_name, agent_availability, agency_name_v2, cian_user_id, is_chats_enabled, company_name, experience, is_sub_agent, user_type, agent_account_type, is_agent, agent_avatar_url, is_hidden, cian_profile_status, agent_reviews)

    def to_dict(self) -> dict:
        result: dict = {}
        result["phoneNumbers"] = from_union([lambda x: from_list(lambda x: to_class(Phone, x), x), from_none], self.phone_numbers)
        result["userTrustLevel"] = from_union([lambda x: to_enum(UserTrustLevel, x), from_none], self.user_trust_level)
        result["agencyName"] = from_union([from_str, from_none], self.agency_name)
        result["agentAvailability"] = from_union([lambda x: to_class(AgentAvailability, x), from_none], self.agent_availability)
        result["agencyNameV2"] = from_union([from_str, from_none], self.agency_name_v2)
        result["cianUserId"] = from_union([from_int, from_none], self.cian_user_id)
        result["isChatsEnabled"] = from_union([from_bool, from_none], self.is_chats_enabled)
        result["companyName"] = from_union([from_str, from_none], self.company_name)
        result["experience"] = from_union([from_str, from_none], self.experience)
        result["isSubAgent"] = from_union([from_bool, from_none], self.is_sub_agent)
        result["userType"] = from_union([lambda x: to_enum(UserType, x), from_none], self.user_type)
        result["agentAccountType"] = from_union([lambda x: to_enum(AgentAccountType, x), from_none], self.agent_account_type)
        result["isAgent"] = from_union([from_bool, from_none], self.is_agent)
        result["agentAvatarUrl"] = from_union([from_str, from_none], self.agent_avatar_url)
        result["isHidden"] = from_union([from_bool, from_none], self.is_hidden)
        result["cianProfileStatus"] = from_union([lambda x: to_enum(CianProfileStatus, x), from_none], self.cian_profile_status)
        result["agentReviews"] = from_union([lambda x: to_class(AgentReviews, x), from_none], self.agent_reviews)
        return result


@dataclass
class ValueAddedServices:
    is_premium: Optional[bool]
    is_paid: Optional[bool]
    is_top3: Optional[bool]
    is_colorized: Optional[bool]
    is_free: Optional[bool]
    is_calltracking: Optional[bool]

    @staticmethod
    def from_dict(obj: Any) -> 'ValueAddedServices':
        assert isinstance(obj, dict)
        is_premium = from_union([from_bool, from_none], obj.get("isPremium"))
        is_paid = from_union([from_bool, from_none], obj.get("isPaid"))
        is_top3 = from_union([from_bool, from_none], obj.get("isTop3"))
        is_colorized = from_union([from_bool, from_none], obj.get("isColorized"))
        is_free = from_union([from_bool, from_none], obj.get("isFree"))
        is_calltracking = from_union([from_bool, from_none], obj.get("isCalltracking"))
        return ValueAddedServices(is_premium, is_paid, is_top3, is_colorized, is_free, is_calltracking)

    def to_dict(self) -> dict:
        result: dict = {}
        result["isPremium"] = from_union([from_bool, from_none], self.is_premium)
        result["isPaid"] = from_union([from_bool, from_none], self.is_paid)
        result["isTop3"] = from_union([from_bool, from_none], self.is_top3)
        result["isColorized"] = from_union([from_bool, from_none], self.is_colorized)
        result["isFree"] = from_union([from_bool, from_none], self.is_free)
        result["isCalltracking"] = from_union([from_bool, from_none], self.is_calltracking)
        return result


@dataclass
class OffersSerialized:
    tax_number: None
    has_phone: None
    newbuilding: Optional[Newbuilding]
    has_hairdryer: None
    room_type: None
    drainage_type: None
    furniture_presence: None
    has_furniture: Optional[bool]
    has_conditioner: None
    share_amount: None
    price_per_unit_area_per_year_rur_for_search: None
    archived_date: None
    layout: Optional[str]
    settlement_name: None
    title: None
    electricity: Optional[Electricity]
    work_time_info: None
    min_price_total: None
    all_from_offrep: None
    platform: Optional[Platform]
    has_extinguishing_system: None
    electricity_power: None
    garage: None
    chat_id: None
    user_id: Optional[int]
    price_total_rur: Optional[int]
    description_minhash: Optional[str]
    has_pool: None
    ga_label: Optional[str]
    total_area: Optional[str]
    leisure: None
    price_per_unit_area: Optional[int]
    separate_wcs_count: None
    property_type: None
    drainage_capacity: None
    geo: Optional[OffersSerializedGeo]
    furniture: None
    all_rooms_area: None
    demand_json_query: Optional[DemandJSONQuery]
    is_paid: Optional[bool]
    name: None
    notes: Optional[Notes]
    children_allowed: None
    price_per_unit_area_per_year_rur: None
    has_check_in_24: None
    min_vat_price_total_per_month_rur: None
    demolished_in_moscow_programm: None
    permitted_use_type: None
    drainage: None
    min_vat_price_total_rur: None
    floor_material_type: None
    vas_type: None
    price_total_usd: Optional[int]
    rosreestr_check: None
    has_lift: None
    phones: Optional[List[Phone]]
    floor_number: Optional[int]
    min_area: None
    is_pro: Optional[bool]
    has_investment_project: None
    statistic: Optional[Statistic]
    price_per_unit_area_usd: Optional[int]
    has_transport_services: None
    price_per_unit_area_rur: Optional[int]
    category: Optional[Category]
    room_area: None
    has_shower: None
    is_colorized: Optional[bool]
    is_hidden_by_user: Optional[bool]
    bedrooms_count: None
    min_price_total_rur: None
    price_total_per_month: None
    external_id: None
    is_occupied: Optional[bool]
    external_url: None
    has_dishwasher: None
    booking: None
    price_total_eur: Optional[int]
    has_wired_internet: None
    additional_phone_lines_allowed: None
    is_top3: Optional[bool]
    possible_to_change_permitted_use_type: None
    price_total_per_month_usd: None
    water: None
    pets_allowed: None
    electricity_type: None
    has_fridge: None
    rooms_area: None
    max_area: None
    has_transfer: None
    publish_terms: Optional[PublishTerms]
    min_price_total_per_month: None
    is_enabled_call_tracking: None
    min_price_total_rur_for_search: None
    gas_type: None
    has_heating: None
    cpl_moderation: None
    water_capacity: None
    flat_type: None
    has_jacuzzi: None
    placement_type: Optional[PlacementType]
    is_fairplay: Optional[bool]
    gas_pressure: None
    has_parking: None
    rent_by_parts_description: None
    has_internet: None
    condition_rating_type: None
    price_per_unit_area_per_month_rur_for_search: None
    categories_ids: Optional[List[int]]
    min_price_total_per_month_rur: None
    feedbox_multi_offer_key: None
    has_bathhouse: None
    has_encumbrances: None
    kitchen_area: None
    is_legal_address_provided: Optional[bool]
    videos: Optional[List[Any]]
    is_favorite: Optional[bool]
    price_per_unit_area_per_month: None
    windows_view_type: None
    has_washer: None
    gas_capacity: None
    cian_id: Optional[int]
    without_client_fee: None
    price_per_unit_area_per_year_eur: None
    cian_user_id: Optional[int]
    phone_lines_count: None
    is_rent_by_parts: Optional[bool]
    loggias_count: None
    has_security: None
    price_per_unit_area_per_year_usd: None
    is_customs: None
    price_changes: Optional[List[PriceChange]]
    business_shopping_center: Optional[BusinessShoppingCenter]
    gas: None
    specialty: Optional[OffersSerializedSpecialty]
    price_per_unit_area_per_year: None
    photos: Optional[List[Photo]]
    has_shop_windows: Optional[bool]
    rooms_for_sale_count: None
    is_penthouse: None
    has_equipment: None
    is_auction: Optional[bool]
    offer_type: Optional[OfferType]
    is_new: Optional[bool]
    vat_price_total_rur: Optional[int]
    edit_date: Optional[datetime]
    price_total_per_month_eur: None
    object_guid: Optional[UUID]
    wc_location_type: None
    has_light: None
    is_from_builder: Optional[bool]
    is_resale_property: None
    repair_type: None
    value_added_services: Optional[ValueAddedServices]
    has_seconded_docs: None
    deal_type: Optional[DealTypeEnum]
    similar: None
    has_drainage: None
    price_total_rur_for_search: Optional[int]
    price_total_per_month_rur: None
    has_kitchen_furniture: None
    is_by_homeowner: None
    has_tv: None
    driveway_type: None
    min_price_total_per_month_rur_for_search: None
    jk_url: Optional[str]
    layout_photo: None
    decoration: None
    has_tea_coffee_sugar: None
    has_electricity: None
    price_per_unit_area_eur: Optional[int]
    call_tracking_provider: None
    input_type: Optional[str]
    has_bathtub: None
    has_wifi: None
    condition_type: Optional[ConditionType]
    water_type: None
    has_garage: None
    can_parts: Optional[bool]
    has_excursions: None
    access_type: None
    is_duplicated_description: Optional[bool]
    id: Optional[int]
    price_per_unit_area_per_month_rur: None
    has_gas: None
    estate_type: None
    project_declaration_url: None
    is_apartments: None
    has_iron: None
    has_water: None
    status: Optional[Status]
    available_from: Optional[datetime]
    is_premium: Optional[bool]
    added: Optional[str]
    description: Optional[str]
    price_total_per_month_rur_for_search: None
    is_in_hidden_base: Optional[bool]
    price_per_unit_area_rur_for_search: Optional[int]
    monthly_income: None
    district: None
    user: Optional[User]
    bargain_terms: Optional[BargainTerms]
    from_developer: None
    has_safe_custody: None
    balconies_count: None
    building: Optional[Building]
    land: Optional[Land]
    living_area: None
    published_user_id: Optional[int]
    beds_count: None
    water_pipes_count: Optional[int]
    vat_price_total_per_month_rur: None
    wc_type: None
    kp: None
    flags: Optional[Flags]
    has_ramp: None
    promo_info: None
    rooms_count: None
    price_total: Optional[int]
    combined_wcs_count: None

    @staticmethod
    def from_dict(obj: Any) -> 'OffersSerialized':
        assert isinstance(obj, dict)
        tax_number = from_none(obj.get("taxNumber"))
        has_phone = from_none(obj.get("hasPhone"))
        newbuilding = from_union([from_none, Newbuilding.from_dict], obj.get("newbuilding"))
        has_hairdryer = from_none(obj.get("hasHairdryer"))
        room_type = from_none(obj.get("roomType"))
        drainage_type = from_none(obj.get("drainageType"))
        furniture_presence = from_none(obj.get("furniturePresence"))
        has_furniture = from_union([from_bool, from_none], obj.get("hasFurniture"))
        has_conditioner = from_none(obj.get("hasConditioner"))
        share_amount = from_none(obj.get("shareAmount"))
        price_per_unit_area_per_year_rur_for_search = from_none(obj.get("pricePerUnitAreaPerYearRurForSearch"))
        archived_date = from_none(obj.get("archivedDate"))
        layout = from_union([from_none, from_str], obj.get("layout"))
        settlement_name = from_none(obj.get("settlementName"))
        title = from_none(obj.get("title"))
        electricity = from_union([from_none, Electricity.from_dict], obj.get("electricity"))
        work_time_info = from_none(obj.get("workTimeInfo"))
        min_price_total = from_none(obj.get("minPriceTotal"))
        all_from_offrep = from_none(obj.get("allFromOffrep"))
        platform = from_union([from_none, Platform.from_dict], obj.get("platform"))
        has_extinguishing_system = from_none(obj.get("hasExtinguishingSystem"))
        electricity_power = from_none(obj.get("electricityPower"))
        garage = from_none(obj.get("garage"))
        chat_id = from_none(obj.get("chatId"))
        user_id = from_union([from_int, from_none], obj.get("userId"))
        price_total_rur = from_union([from_int, from_none], obj.get("priceTotalRur"))
        description_minhash = from_union([from_str, from_none], obj.get("descriptionMinhash"))
        has_pool = from_none(obj.get("hasPool"))
        ga_label = from_union([from_str, from_none], obj.get("gaLabel"))
        total_area = from_union([from_str, from_none], obj.get("totalArea"))
        leisure = from_none(obj.get("leisure"))
        price_per_unit_area = from_union([from_int, from_none], obj.get("pricePerUnitArea"))
        separate_wcs_count = from_none(obj.get("separateWcsCount"))
        property_type = from_none(obj.get("propertyType"))
        drainage_capacity = from_none(obj.get("drainageCapacity"))
        geo = from_union([OffersSerializedGeo.from_dict, from_none], obj.get("geo"))
        furniture = from_none(obj.get("furniture"))
        all_rooms_area = from_none(obj.get("allRoomsArea"))
        demand_json_query = from_union([DemandJSONQuery.from_dict, from_none], obj.get("demandJsonQuery"))
        is_paid = from_union([from_bool, from_none], obj.get("isPaid"))
        name = from_none(obj.get("name"))
        notes = from_union([Notes.from_dict, from_none], obj.get("notes"))
        children_allowed = from_none(obj.get("childrenAllowed"))
        price_per_unit_area_per_year_rur = from_none(obj.get("pricePerUnitAreaPerYearRur"))
        has_check_in_24 = from_none(obj.get("hasCheckIn_24"))
        min_vat_price_total_per_month_rur = from_none(obj.get("minVatPriceTotalPerMonthRur"))
        demolished_in_moscow_programm = from_none(obj.get("demolishedInMoscowProgramm"))
        permitted_use_type = from_none(obj.get("permittedUseType"))
        drainage = from_none(obj.get("drainage"))
        min_vat_price_total_rur = from_none(obj.get("minVatPriceTotalRur"))
        floor_material_type = from_none(obj.get("floorMaterialType"))
        vas_type = from_none(obj.get("vasType"))
        price_total_usd = from_union([from_int, from_none], obj.get("priceTotalUsd"))
        rosreestr_check = from_none(obj.get("rosreestrCheck"))
        has_lift = from_none(obj.get("hasLift"))
        phones = from_union([lambda x: from_list(Phone.from_dict, x), from_none], obj.get("phones"))
        floor_number = from_union([from_int, from_none], obj.get("floorNumber"))
        min_area = from_none(obj.get("minArea"))
        is_pro = from_union([from_bool, from_none], obj.get("isPro"))
        has_investment_project = from_none(obj.get("hasInvestmentProject"))
        statistic = from_union([Statistic.from_dict, from_none], obj.get("statistic"))
        price_per_unit_area_usd = from_union([from_int, from_none], obj.get("pricePerUnitAreaUsd"))
        has_transport_services = from_none(obj.get("hasTransportServices"))
        price_per_unit_area_rur = from_union([from_int, from_none], obj.get("pricePerUnitAreaRur"))
        category = from_union([Category, from_none], obj.get("category"))
        room_area = from_none(obj.get("roomArea"))
        has_shower = from_none(obj.get("hasShower"))
        is_colorized = from_union([from_bool, from_none], obj.get("isColorized"))
        is_hidden_by_user = from_union([from_bool, from_none], obj.get("isHiddenByUser"))
        bedrooms_count = from_none(obj.get("bedroomsCount"))
        min_price_total_rur = from_none(obj.get("minPriceTotalRur"))
        price_total_per_month = from_none(obj.get("priceTotalPerMonth"))
        external_id = from_none(obj.get("externalId"))
        is_occupied = from_union([from_bool, from_none], obj.get("isOccupied"))
        external_url = from_none(obj.get("externalUrl"))
        has_dishwasher = from_none(obj.get("hasDishwasher"))
        booking = from_none(obj.get("booking"))
        price_total_eur = from_union([from_int, from_none], obj.get("priceTotalEur"))
        has_wired_internet = from_none(obj.get("hasWiredInternet"))
        additional_phone_lines_allowed = from_none(obj.get("additionalPhoneLinesAllowed"))
        is_top3 = from_union([from_bool, from_none], obj.get("isTop3"))
        possible_to_change_permitted_use_type = from_none(obj.get("possibleToChangePermittedUseType"))
        price_total_per_month_usd = from_none(obj.get("priceTotalPerMonthUsd"))
        water = from_none(obj.get("water"))
        pets_allowed = from_none(obj.get("petsAllowed"))
        electricity_type = from_none(obj.get("electricityType"))
        has_fridge = from_none(obj.get("hasFridge"))
        rooms_area = from_none(obj.get("roomsArea"))
        max_area = from_none(obj.get("maxArea"))
        has_transfer = from_none(obj.get("hasTransfer"))
        publish_terms = from_union([PublishTerms.from_dict, from_none], obj.get("publishTerms"))
        min_price_total_per_month = from_none(obj.get("minPriceTotalPerMonth"))
        is_enabled_call_tracking = from_none(obj.get("isEnabledCallTracking"))
        min_price_total_rur_for_search = from_none(obj.get("minPriceTotalRurForSearch"))
        gas_type = from_none(obj.get("gasType"))
        has_heating = from_none(obj.get("hasHeating"))
        cpl_moderation = from_none(obj.get("cplModeration"))
        water_capacity = from_none(obj.get("waterCapacity"))
        flat_type = from_none(obj.get("flatType"))
        has_jacuzzi = from_none(obj.get("hasJacuzzi"))
        placement_type = from_union([from_none, PlacementType], obj.get("placementType"))
        is_fairplay = from_union([from_bool, from_none], obj.get("isFairplay"))
        gas_pressure = from_none(obj.get("gasPressure"))
        has_parking = from_none(obj.get("hasParking"))
        rent_by_parts_description = from_none(obj.get("rentByPartsDescription"))
        has_internet = from_none(obj.get("hasInternet"))
        condition_rating_type = from_none(obj.get("conditionRatingType"))
        price_per_unit_area_per_month_rur_for_search = from_none(obj.get("pricePerUnitAreaPerMonthRurForSearch"))
        categories_ids = from_union([lambda x: from_list(from_int, x), from_none], obj.get("categoriesIds"))
        min_price_total_per_month_rur = from_none(obj.get("minPriceTotalPerMonthRur"))
        feedbox_multi_offer_key = from_none(obj.get("feedboxMultiOfferKey"))
        has_bathhouse = from_none(obj.get("hasBathhouse"))
        has_encumbrances = from_none(obj.get("hasEncumbrances"))
        kitchen_area = from_none(obj.get("kitchenArea"))
        is_legal_address_provided = from_union([from_bool, from_none], obj.get("isLegalAddressProvided"))
        videos = from_union([lambda x: from_list(lambda x: x, x), from_none], obj.get("videos"))
        is_favorite = from_union([from_bool, from_none], obj.get("isFavorite"))
        price_per_unit_area_per_month = from_none(obj.get("pricePerUnitAreaPerMonth"))
        windows_view_type = from_none(obj.get("windowsViewType"))
        has_washer = from_none(obj.get("hasWasher"))
        gas_capacity = from_none(obj.get("gasCapacity"))
        cian_id = from_union([from_int, from_none], obj.get("cianId"))
        without_client_fee = from_none(obj.get("withoutClientFee"))
        price_per_unit_area_per_year_eur = from_none(obj.get("pricePerUnitAreaPerYearEur"))
        cian_user_id = from_union([from_int, from_none], obj.get("cianUserId"))
        phone_lines_count = from_none(obj.get("phoneLinesCount"))
        is_rent_by_parts = from_union([from_bool, from_none], obj.get("isRentByParts"))
        loggias_count = from_none(obj.get("loggiasCount"))
        has_security = from_none(obj.get("hasSecurity"))
        price_per_unit_area_per_year_usd = from_none(obj.get("pricePerUnitAreaPerYearUsd"))
        is_customs = from_none(obj.get("isCustoms"))
        price_changes = from_union([lambda x: from_list(PriceChange.from_dict, x), from_none], obj.get("priceChanges"))
        business_shopping_center = from_union([from_none, BusinessShoppingCenter.from_dict], obj.get("businessShoppingCenter"))
        gas = from_none(obj.get("gas"))
        specialty = from_union([OffersSerializedSpecialty.from_dict, from_none], obj.get("specialty"))
        price_per_unit_area_per_year = from_none(obj.get("pricePerUnitAreaPerYear"))
        photos = from_union([lambda x: from_list(Photo.from_dict, x), from_none], obj.get("photos"))
        has_shop_windows = from_union([from_bool, from_none], obj.get("hasShopWindows"))
        rooms_for_sale_count = from_none(obj.get("roomsForSaleCount"))
        is_penthouse = from_none(obj.get("isPenthouse"))
        has_equipment = from_none(obj.get("hasEquipment"))
        is_auction = from_union([from_bool, from_none], obj.get("isAuction"))
        offer_type = from_union([OfferType, from_none], obj.get("offerType"))
        is_new = from_union([from_bool, from_none], obj.get("isNew"))
        vat_price_total_rur = from_union([from_int, from_none], obj.get("vatPriceTotalRur"))
        edit_date = from_union([from_datetime, from_none], obj.get("editDate"))
        price_total_per_month_eur = from_none(obj.get("priceTotalPerMonthEur"))
        object_guid = from_union([lambda x: UUID(x), from_none], obj.get("objectGuid"))
        wc_location_type = from_none(obj.get("wcLocationType"))
        has_light = from_none(obj.get("hasLight"))
        is_from_builder = from_union([from_bool, from_none], obj.get("isFromBuilder"))
        is_resale_property = from_none(obj.get("isResaleProperty"))
        repair_type = from_none(obj.get("repairType"))
        value_added_services = from_union([ValueAddedServices.from_dict, from_none], obj.get("valueAddedServices"))
        has_seconded_docs = from_none(obj.get("hasSecondedDocs"))
        deal_type = from_union([DealTypeEnum, from_none], obj.get("dealType"))
        similar = from_none(obj.get("similar"))
        has_drainage = from_none(obj.get("hasDrainage"))
        price_total_rur_for_search = from_union([from_int, from_none], obj.get("priceTotalRurForSearch"))
        price_total_per_month_rur = from_none(obj.get("priceTotalPerMonthRur"))
        has_kitchen_furniture = from_none(obj.get("hasKitchenFurniture"))
        is_by_homeowner = from_none(obj.get("isByHomeowner"))
        has_tv = from_none(obj.get("hasTv"))
        driveway_type = from_none(obj.get("drivewayType"))
        min_price_total_per_month_rur_for_search = from_none(obj.get("minPriceTotalPerMonthRurForSearch"))
        jk_url = from_union([from_none, from_str], obj.get("jkUrl"))
        layout_photo = from_none(obj.get("layoutPhoto"))
        decoration = from_none(obj.get("decoration"))
        has_tea_coffee_sugar = from_none(obj.get("hasTeaCoffeeSugar"))
        has_electricity = from_none(obj.get("hasElectricity"))
        price_per_unit_area_eur = from_union([from_int, from_none], obj.get("pricePerUnitAreaEur"))
        call_tracking_provider = from_none(obj.get("callTrackingProvider"))
        input_type = from_union([from_none, from_str], obj.get("inputType"))
        has_bathtub = from_none(obj.get("hasBathtub"))
        has_wifi = from_none(obj.get("hasWifi"))
        condition_type = from_union([from_none, ConditionType], obj.get("conditionType"))
        water_type = from_none(obj.get("waterType"))
        has_garage = from_none(obj.get("hasGarage"))
        can_parts = from_union([from_bool, from_none], obj.get("canParts"))
        has_excursions = from_none(obj.get("hasExcursions"))
        access_type = from_none(obj.get("accessType"))
        is_duplicated_description = from_union([from_bool, from_none], obj.get("isDuplicatedDescription"))
        id = from_union([from_int, from_none], obj.get("id"))
        price_per_unit_area_per_month_rur = from_none(obj.get("pricePerUnitAreaPerMonthRur"))
        has_gas = from_none(obj.get("hasGas"))
        estate_type = from_none(obj.get("estateType"))
        project_declaration_url = from_none(obj.get("projectDeclarationUrl"))
        is_apartments = from_none(obj.get("isApartments"))
        has_iron = from_none(obj.get("hasIron"))
        has_water = from_none(obj.get("hasWater"))
        status = from_union([Status, from_none], obj.get("status"))
        available_from = from_union([from_datetime, from_none], obj.get("availableFrom"))
        is_premium = from_union([from_bool, from_none], obj.get("isPremium"))
        added = from_union([from_str, from_none], obj.get("added"))
        description = from_union([from_str, from_none], obj.get("description"))
        price_total_per_month_rur_for_search = from_none(obj.get("priceTotalPerMonthRurForSearch"))
        is_in_hidden_base = from_union([from_bool, from_none], obj.get("isInHiddenBase"))
        price_per_unit_area_rur_for_search = from_union([from_int, from_none], obj.get("pricePerUnitAreaRurForSearch"))
        monthly_income = from_none(obj.get("monthlyIncome"))
        district = from_none(obj.get("district"))
        user = from_union([User.from_dict, from_none], obj.get("user"))
        bargain_terms = from_union([BargainTerms.from_dict, from_none], obj.get("bargainTerms"))
        from_developer = from_none(obj.get("fromDeveloper"))
        has_safe_custody = from_none(obj.get("hasSafeCustody"))
        balconies_count = from_none(obj.get("balconiesCount"))
        building = from_union([Building.from_dict, from_none], obj.get("building"))
        land = from_union([from_none, Land.from_dict], obj.get("land"))
        living_area = from_none(obj.get("livingArea"))
        published_user_id = from_union([from_int, from_none], obj.get("publishedUserId"))
        beds_count = from_none(obj.get("bedsCount"))
        water_pipes_count = from_union([from_int, from_none], obj.get("waterPipesCount"))
        vat_price_total_per_month_rur = from_none(obj.get("vatPriceTotalPerMonthRur"))
        wc_type = from_none(obj.get("wcType"))
        kp = from_none(obj.get("kp"))
        flags = from_union([Flags.from_dict, from_none], obj.get("flags"))
        has_ramp = from_none(obj.get("hasRamp"))
        promo_info = from_none(obj.get("promoInfo"))
        rooms_count = from_none(obj.get("roomsCount"))
        price_total = from_union([from_int, from_none], obj.get("priceTotal"))
        combined_wcs_count = from_none(obj.get("combinedWcsCount"))
        return OffersSerialized(tax_number, has_phone, newbuilding, has_hairdryer, room_type, drainage_type, furniture_presence, has_furniture, has_conditioner, share_amount, price_per_unit_area_per_year_rur_for_search, archived_date, layout, settlement_name, title, electricity, work_time_info, min_price_total, all_from_offrep, platform, has_extinguishing_system, electricity_power, garage, chat_id, user_id, price_total_rur, description_minhash, has_pool, ga_label, total_area, leisure, price_per_unit_area, separate_wcs_count, property_type, drainage_capacity, geo, furniture, all_rooms_area, demand_json_query, is_paid, name, notes, children_allowed, price_per_unit_area_per_year_rur, has_check_in_24, min_vat_price_total_per_month_rur, demolished_in_moscow_programm, permitted_use_type, drainage, min_vat_price_total_rur, floor_material_type, vas_type, price_total_usd, rosreestr_check, has_lift, phones, floor_number, min_area, is_pro, has_investment_project, statistic, price_per_unit_area_usd, has_transport_services, price_per_unit_area_rur, category, room_area, has_shower, is_colorized, is_hidden_by_user, bedrooms_count, min_price_total_rur, price_total_per_month, external_id, is_occupied, external_url, has_dishwasher, booking, price_total_eur, has_wired_internet, additional_phone_lines_allowed, is_top3, possible_to_change_permitted_use_type, price_total_per_month_usd, water, pets_allowed, electricity_type, has_fridge, rooms_area, max_area, has_transfer, publish_terms, min_price_total_per_month, is_enabled_call_tracking, min_price_total_rur_for_search, gas_type, has_heating, cpl_moderation, water_capacity, flat_type, has_jacuzzi, placement_type, is_fairplay, gas_pressure, has_parking, rent_by_parts_description, has_internet, condition_rating_type, price_per_unit_area_per_month_rur_for_search, categories_ids, min_price_total_per_month_rur, feedbox_multi_offer_key, has_bathhouse, has_encumbrances, kitchen_area, is_legal_address_provided, videos, is_favorite, price_per_unit_area_per_month, windows_view_type, has_washer, gas_capacity, cian_id, without_client_fee, price_per_unit_area_per_year_eur, cian_user_id, phone_lines_count, is_rent_by_parts, loggias_count, has_security, price_per_unit_area_per_year_usd, is_customs, price_changes, business_shopping_center, gas, specialty, price_per_unit_area_per_year, photos, has_shop_windows, rooms_for_sale_count, is_penthouse, has_equipment, is_auction, offer_type, is_new, vat_price_total_rur, edit_date, price_total_per_month_eur, object_guid, wc_location_type, has_light, is_from_builder, is_resale_property, repair_type, value_added_services, has_seconded_docs, deal_type, similar, has_drainage, price_total_rur_for_search, price_total_per_month_rur, has_kitchen_furniture, is_by_homeowner, has_tv, driveway_type, min_price_total_per_month_rur_for_search, jk_url, layout_photo, decoration, has_tea_coffee_sugar, has_electricity, price_per_unit_area_eur, call_tracking_provider, input_type, has_bathtub, has_wifi, condition_type, water_type, has_garage, can_parts, has_excursions, access_type, is_duplicated_description, id, price_per_unit_area_per_month_rur, has_gas, estate_type, project_declaration_url, is_apartments, has_iron, has_water, status, available_from, is_premium, added, description, price_total_per_month_rur_for_search, is_in_hidden_base, price_per_unit_area_rur_for_search, monthly_income, district, user, bargain_terms, from_developer, has_safe_custody, balconies_count, building, land, living_area, published_user_id, beds_count, water_pipes_count, vat_price_total_per_month_rur, wc_type, kp, flags, has_ramp, promo_info, rooms_count, price_total, combined_wcs_count)

    def to_dict(self) -> dict:
        result: dict = {}
        result["taxNumber"] = from_none(self.tax_number)
        result["hasPhone"] = from_none(self.has_phone)
        result["newbuilding"] = from_union([from_none, lambda x: to_class(Newbuilding, x)], self.newbuilding)
        result["hasHairdryer"] = from_none(self.has_hairdryer)
        result["roomType"] = from_none(self.room_type)
        result["drainageType"] = from_none(self.drainage_type)
        result["furniturePresence"] = from_none(self.furniture_presence)
        result["hasFurniture"] = from_union([from_bool, from_none], self.has_furniture)
        result["hasConditioner"] = from_none(self.has_conditioner)
        result["shareAmount"] = from_none(self.share_amount)
        result["pricePerUnitAreaPerYearRurForSearch"] = from_none(self.price_per_unit_area_per_year_rur_for_search)
        result["archivedDate"] = from_none(self.archived_date)
        result["layout"] = from_union([from_none, from_str], self.layout)
        result["settlementName"] = from_none(self.settlement_name)
        result["title"] = from_none(self.title)
        result["electricity"] = from_union([from_none, lambda x: to_class(Electricity, x)], self.electricity)
        result["workTimeInfo"] = from_none(self.work_time_info)
        result["minPriceTotal"] = from_none(self.min_price_total)
        result["allFromOffrep"] = from_none(self.all_from_offrep)
        result["platform"] = from_union([from_none, lambda x: to_class(Platform, x)], self.platform)
        result["hasExtinguishingSystem"] = from_none(self.has_extinguishing_system)
        result["electricityPower"] = from_none(self.electricity_power)
        result["garage"] = from_none(self.garage)
        result["chatId"] = from_none(self.chat_id)
        result["userId"] = from_union([from_int, from_none], self.user_id)
        result["priceTotalRur"] = from_union([from_int, from_none], self.price_total_rur)
        result["descriptionMinhash"] = from_union([from_str, from_none], self.description_minhash)
        result["hasPool"] = from_none(self.has_pool)
        result["gaLabel"] = from_union([from_str, from_none], self.ga_label)
        result["totalArea"] = from_union([from_str, from_none], self.total_area)
        result["leisure"] = from_none(self.leisure)
        result["pricePerUnitArea"] = from_union([from_int, from_none], self.price_per_unit_area)
        result["separateWcsCount"] = from_none(self.separate_wcs_count)
        result["propertyType"] = from_none(self.property_type)
        result["drainageCapacity"] = from_none(self.drainage_capacity)
        result["geo"] = from_union([lambda x: to_class(OffersSerializedGeo, x), from_none], self.geo)
        result["furniture"] = from_none(self.furniture)
        result["allRoomsArea"] = from_none(self.all_rooms_area)
        result["demandJsonQuery"] = from_union([lambda x: to_class(DemandJSONQuery, x), from_none], self.demand_json_query)
        result["isPaid"] = from_union([from_bool, from_none], self.is_paid)
        result["name"] = from_none(self.name)
        result["notes"] = from_union([lambda x: to_class(Notes, x), from_none], self.notes)
        result["childrenAllowed"] = from_none(self.children_allowed)
        result["pricePerUnitAreaPerYearRur"] = from_none(self.price_per_unit_area_per_year_rur)
        result["hasCheckIn_24"] = from_none(self.has_check_in_24)
        result["minVatPriceTotalPerMonthRur"] = from_none(self.min_vat_price_total_per_month_rur)
        result["demolishedInMoscowProgramm"] = from_none(self.demolished_in_moscow_programm)
        result["permittedUseType"] = from_none(self.permitted_use_type)
        result["drainage"] = from_none(self.drainage)
        result["minVatPriceTotalRur"] = from_none(self.min_vat_price_total_rur)
        result["floorMaterialType"] = from_none(self.floor_material_type)
        result["vasType"] = from_none(self.vas_type)
        result["priceTotalUsd"] = from_union([from_int, from_none], self.price_total_usd)
        result["rosreestrCheck"] = from_none(self.rosreestr_check)
        result["hasLift"] = from_none(self.has_lift)
        result["phones"] = from_union([lambda x: from_list(lambda x: to_class(Phone, x), x), from_none], self.phones)
        result["floorNumber"] = from_union([from_int, from_none], self.floor_number)
        result["minArea"] = from_none(self.min_area)
        result["isPro"] = from_union([from_bool, from_none], self.is_pro)
        result["hasInvestmentProject"] = from_none(self.has_investment_project)
        result["statistic"] = from_union([lambda x: to_class(Statistic, x), from_none], self.statistic)
        result["pricePerUnitAreaUsd"] = from_union([from_int, from_none], self.price_per_unit_area_usd)
        result["hasTransportServices"] = from_none(self.has_transport_services)
        result["pricePerUnitAreaRur"] = from_union([from_int, from_none], self.price_per_unit_area_rur)
        result["category"] = from_union([lambda x: to_enum(Category, x), from_none], self.category)
        result["roomArea"] = from_none(self.room_area)
        result["hasShower"] = from_none(self.has_shower)
        result["isColorized"] = from_union([from_bool, from_none], self.is_colorized)
        result["isHiddenByUser"] = from_union([from_bool, from_none], self.is_hidden_by_user)
        result["bedroomsCount"] = from_none(self.bedrooms_count)
        result["minPriceTotalRur"] = from_none(self.min_price_total_rur)
        result["priceTotalPerMonth"] = from_none(self.price_total_per_month)
        result["externalId"] = from_none(self.external_id)
        result["isOccupied"] = from_union([from_bool, from_none], self.is_occupied)
        result["externalUrl"] = from_none(self.external_url)
        result["hasDishwasher"] = from_none(self.has_dishwasher)
        result["booking"] = from_none(self.booking)
        result["priceTotalEur"] = from_union([from_int, from_none], self.price_total_eur)
        result["hasWiredInternet"] = from_none(self.has_wired_internet)
        result["additionalPhoneLinesAllowed"] = from_none(self.additional_phone_lines_allowed)
        result["isTop3"] = from_union([from_bool, from_none], self.is_top3)
        result["possibleToChangePermittedUseType"] = from_none(self.possible_to_change_permitted_use_type)
        result["priceTotalPerMonthUsd"] = from_none(self.price_total_per_month_usd)
        result["water"] = from_none(self.water)
        result["petsAllowed"] = from_none(self.pets_allowed)
        result["electricityType"] = from_none(self.electricity_type)
        result["hasFridge"] = from_none(self.has_fridge)
        result["roomsArea"] = from_none(self.rooms_area)
        result["maxArea"] = from_none(self.max_area)
        result["hasTransfer"] = from_none(self.has_transfer)
        result["publishTerms"] = from_union([lambda x: to_class(PublishTerms, x), from_none], self.publish_terms)
        result["minPriceTotalPerMonth"] = from_none(self.min_price_total_per_month)
        result["isEnabledCallTracking"] = from_none(self.is_enabled_call_tracking)
        result["minPriceTotalRurForSearch"] = from_none(self.min_price_total_rur_for_search)
        result["gasType"] = from_none(self.gas_type)
        result["hasHeating"] = from_none(self.has_heating)
        result["cplModeration"] = from_none(self.cpl_moderation)
        result["waterCapacity"] = from_none(self.water_capacity)
        result["flatType"] = from_none(self.flat_type)
        result["hasJacuzzi"] = from_none(self.has_jacuzzi)
        result["placementType"] = from_union([from_none, lambda x: to_enum(PlacementType, x)], self.placement_type)
        result["isFairplay"] = from_union([from_bool, from_none], self.is_fairplay)
        result["gasPressure"] = from_none(self.gas_pressure)
        result["hasParking"] = from_none(self.has_parking)
        result["rentByPartsDescription"] = from_none(self.rent_by_parts_description)
        result["hasInternet"] = from_none(self.has_internet)
        result["conditionRatingType"] = from_none(self.condition_rating_type)
        result["pricePerUnitAreaPerMonthRurForSearch"] = from_none(self.price_per_unit_area_per_month_rur_for_search)
        result["categoriesIds"] = from_union([lambda x: from_list(from_int, x), from_none], self.categories_ids)
        result["minPriceTotalPerMonthRur"] = from_none(self.min_price_total_per_month_rur)
        result["feedboxMultiOfferKey"] = from_none(self.feedbox_multi_offer_key)
        result["hasBathhouse"] = from_none(self.has_bathhouse)
        result["hasEncumbrances"] = from_none(self.has_encumbrances)
        result["kitchenArea"] = from_none(self.kitchen_area)
        result["isLegalAddressProvided"] = from_union([from_bool, from_none], self.is_legal_address_provided)
        result["videos"] = from_union([lambda x: from_list(lambda x: x, x), from_none], self.videos)
        result["isFavorite"] = from_union([from_bool, from_none], self.is_favorite)
        result["pricePerUnitAreaPerMonth"] = from_none(self.price_per_unit_area_per_month)
        result["windowsViewType"] = from_none(self.windows_view_type)
        result["hasWasher"] = from_none(self.has_washer)
        result["gasCapacity"] = from_none(self.gas_capacity)
        result["cianId"] = from_union([from_int, from_none], self.cian_id)
        result["withoutClientFee"] = from_none(self.without_client_fee)
        result["pricePerUnitAreaPerYearEur"] = from_none(self.price_per_unit_area_per_year_eur)
        result["cianUserId"] = from_union([from_int, from_none], self.cian_user_id)
        result["phoneLinesCount"] = from_none(self.phone_lines_count)
        result["isRentByParts"] = from_union([from_bool, from_none], self.is_rent_by_parts)
        result["loggiasCount"] = from_none(self.loggias_count)
        result["hasSecurity"] = from_none(self.has_security)
        result["pricePerUnitAreaPerYearUsd"] = from_none(self.price_per_unit_area_per_year_usd)
        result["isCustoms"] = from_none(self.is_customs)
        result["priceChanges"] = from_union([lambda x: from_list(lambda x: to_class(PriceChange, x), x), from_none], self.price_changes)
        result["businessShoppingCenter"] = from_union([from_none, lambda x: to_class(BusinessShoppingCenter, x)], self.business_shopping_center)
        result["gas"] = from_none(self.gas)
        result["specialty"] = from_union([lambda x: to_class(OffersSerializedSpecialty, x), from_none], self.specialty)
        result["pricePerUnitAreaPerYear"] = from_none(self.price_per_unit_area_per_year)
        result["photos"] = from_union([lambda x: from_list(lambda x: to_class(Photo, x), x), from_none], self.photos)
        result["hasShopWindows"] = from_union([from_bool, from_none], self.has_shop_windows)
        result["roomsForSaleCount"] = from_none(self.rooms_for_sale_count)
        result["isPenthouse"] = from_none(self.is_penthouse)
        result["hasEquipment"] = from_none(self.has_equipment)
        result["isAuction"] = from_union([from_bool, from_none], self.is_auction)
        result["offerType"] = from_union([lambda x: to_enum(OfferType, x), from_none], self.offer_type)
        result["isNew"] = from_union([from_bool, from_none], self.is_new)
        result["vatPriceTotalRur"] = from_union([from_int, from_none], self.vat_price_total_rur)
        result["editDate"] = from_union([lambda x: x.isoformat(), from_none], self.edit_date)
        result["priceTotalPerMonthEur"] = from_none(self.price_total_per_month_eur)
        result["objectGuid"] = from_union([lambda x: str(x), from_none], self.object_guid)
        result["wcLocationType"] = from_none(self.wc_location_type)
        result["hasLight"] = from_none(self.has_light)
        result["isFromBuilder"] = from_union([from_bool, from_none], self.is_from_builder)
        result["isResaleProperty"] = from_none(self.is_resale_property)
        result["repairType"] = from_none(self.repair_type)
        result["valueAddedServices"] = from_union([lambda x: to_class(ValueAddedServices, x), from_none], self.value_added_services)
        result["hasSecondedDocs"] = from_none(self.has_seconded_docs)
        result["dealType"] = from_union([lambda x: to_enum(DealTypeEnum, x), from_none], self.deal_type)
        result["similar"] = from_none(self.similar)
        result["hasDrainage"] = from_none(self.has_drainage)
        result["priceTotalRurForSearch"] = from_union([from_int, from_none], self.price_total_rur_for_search)
        result["priceTotalPerMonthRur"] = from_none(self.price_total_per_month_rur)
        result["hasKitchenFurniture"] = from_none(self.has_kitchen_furniture)
        result["isByHomeowner"] = from_none(self.is_by_homeowner)
        result["hasTv"] = from_none(self.has_tv)
        result["drivewayType"] = from_none(self.driveway_type)
        result["minPriceTotalPerMonthRurForSearch"] = from_none(self.min_price_total_per_month_rur_for_search)
        result["jkUrl"] = from_union([from_none, from_str], self.jk_url)
        result["layoutPhoto"] = from_none(self.layout_photo)
        result["decoration"] = from_none(self.decoration)
        result["hasTeaCoffeeSugar"] = from_none(self.has_tea_coffee_sugar)
        result["hasElectricity"] = from_none(self.has_electricity)
        result["pricePerUnitAreaEur"] = from_union([from_int, from_none], self.price_per_unit_area_eur)
        result["callTrackingProvider"] = from_none(self.call_tracking_provider)
        result["inputType"] = from_union([from_none, from_str], self.input_type)
        result["hasBathtub"] = from_none(self.has_bathtub)
        result["hasWifi"] = from_none(self.has_wifi)
        result["conditionType"] = from_union([from_none, lambda x: to_enum(ConditionType, x)], self.condition_type)
        result["waterType"] = from_none(self.water_type)
        result["hasGarage"] = from_none(self.has_garage)
        result["canParts"] = from_union([from_bool, from_none], self.can_parts)
        result["hasExcursions"] = from_none(self.has_excursions)
        result["accessType"] = from_none(self.access_type)
        result["isDuplicatedDescription"] = from_union([from_bool, from_none], self.is_duplicated_description)
        result["id"] = from_union([from_int, from_none], self.id)
        result["pricePerUnitAreaPerMonthRur"] = from_none(self.price_per_unit_area_per_month_rur)
        result["hasGas"] = from_none(self.has_gas)
        result["estateType"] = from_none(self.estate_type)
        result["projectDeclarationUrl"] = from_none(self.project_declaration_url)
        result["isApartments"] = from_none(self.is_apartments)
        result["hasIron"] = from_none(self.has_iron)
        result["hasWater"] = from_none(self.has_water)
        result["status"] = from_union([lambda x: to_enum(Status, x), from_none], self.status)
        result["availableFrom"] = from_union([lambda x: x.isoformat(), from_none], self.available_from)
        result["isPremium"] = from_union([from_bool, from_none], self.is_premium)
        result["added"] = from_union([from_str, from_none], self.added)
        result["description"] = from_union([from_str, from_none], self.description)
        result["priceTotalPerMonthRurForSearch"] = from_none(self.price_total_per_month_rur_for_search)
        result["isInHiddenBase"] = from_union([from_bool, from_none], self.is_in_hidden_base)
        result["pricePerUnitAreaRurForSearch"] = from_union([from_int, from_none], self.price_per_unit_area_rur_for_search)
        result["monthlyIncome"] = from_none(self.monthly_income)
        result["district"] = from_none(self.district)
        result["user"] = from_union([lambda x: to_class(User, x), from_none], self.user)
        result["bargainTerms"] = from_union([lambda x: to_class(BargainTerms, x), from_none], self.bargain_terms)
        result["fromDeveloper"] = from_none(self.from_developer)
        result["hasSafeCustody"] = from_none(self.has_safe_custody)
        result["balconiesCount"] = from_none(self.balconies_count)
        result["building"] = from_union([lambda x: to_class(Building, x), from_none], self.building)
        result["land"] = from_union([from_none, lambda x: to_class(Land, x)], self.land)
        result["livingArea"] = from_none(self.living_area)
        result["publishedUserId"] = from_union([from_int, from_none], self.published_user_id)
        result["bedsCount"] = from_none(self.beds_count)
        result["waterPipesCount"] = from_union([from_int, from_none], self.water_pipes_count)
        result["vatPriceTotalPerMonthRur"] = from_none(self.vat_price_total_per_month_rur)
        result["wcType"] = from_none(self.wc_type)
        result["kp"] = from_none(self.kp)
        result["flags"] = from_union([lambda x: to_class(Flags, x), from_none], self.flags)
        result["hasRamp"] = from_none(self.has_ramp)
        result["promoInfo"] = from_none(self.promo_info)
        result["roomsCount"] = from_none(self.rooms_count)
        result["priceTotal"] = from_union([from_int, from_none], self.price_total)
        result["combinedWcsCount"] = from_none(self.combined_wcs_count)
        return result


@dataclass
class SEOData:
    noindex: Optional[bool]
    h1: Optional[str]
    meta_description: Optional[str]
    description: Optional[str]
    title: Optional[str]

    @staticmethod
    def from_dict(obj: Any) -> 'SEOData':
        assert isinstance(obj, dict)
        noindex = from_union([from_bool, from_none], obj.get("noindex"))
        h1 = from_union([from_str, from_none], obj.get("h1"))
        meta_description = from_union([from_str, from_none], obj.get("metaDescription"))
        description = from_union([from_str, from_none], obj.get("description"))
        title = from_union([from_str, from_none], obj.get("title"))
        return SEOData(noindex, h1, meta_description, description, title)

    def to_dict(self) -> dict:
        result: dict = {}
        result["noindex"] = from_union([from_bool, from_none], self.noindex)
        result["h1"] = from_union([from_str, from_none], self.h1)
        result["metaDescription"] = from_union([from_str, from_none], self.meta_description)
        result["description"] = from_union([from_str, from_none], self.description)
        result["title"] = from_union([from_str, from_none], self.title)
        return result


@dataclass
class Data:
    suggestions_query: None
    json_query: Optional[JSONQuery]
    query_string: Optional[str]
    is_newobjects_enabled: Optional[bool]
    is_call_button_enabled: Optional[bool]
    ml_ranking_guid: None
    breadcrumbs: Optional[List[Breadcrumb]]
    top3_is_on: Optional[bool]
    seo_data: Optional[SEOData]
    offer_count: Optional[int]
    offers_serialized: Optional[List[OffersSerialized]]
    aggregated_count: Optional[int]
    avg_price_informer: None
    call_button_variant: Optional[int]
    search_uuid: None
    newobjects_serialized: Optional[List[Any]]
    ml_ranking_model_version: None

    @staticmethod
    def from_dict(obj: Any) -> 'Data':
        assert isinstance(obj, dict)
        suggestions_query = from_none(obj.get("suggestionsQuery"))
        json_query = from_union([JSONQuery.from_dict, from_none], obj.get("jsonQuery"))
        query_string = from_union([from_str, from_none], obj.get("queryString"))
        is_newobjects_enabled = from_union([from_bool, from_none], obj.get("isNewobjectsEnabled"))
        is_call_button_enabled = from_union([from_bool, from_none], obj.get("isCallButtonEnabled"))
        ml_ranking_guid = from_none(obj.get("mlRankingGuid"))
        breadcrumbs = from_union([lambda x: from_list(Breadcrumb.from_dict, x), from_none], obj.get("breadcrumbs"))
        top3_is_on = from_union([from_bool, from_none], obj.get("top3IsOn"))
        seo_data = from_union([SEOData.from_dict, from_none], obj.get("seoData"))
        offer_count = from_union([from_int, from_none], obj.get("offerCount"))
        offers_serialized = from_union([lambda x: from_list(OffersSerialized.from_dict, x), from_none], obj.get("offersSerialized"))
        aggregated_count = from_union([from_int, from_none], obj.get("aggregatedCount"))
        avg_price_informer = from_none(obj.get("avgPriceInformer"))
        call_button_variant = from_union([from_int, from_none], obj.get("callButtonVariant"))
        search_uuid = from_none(obj.get("searchUuid"))
        newobjects_serialized = from_union([lambda x: from_list(lambda x: x, x), from_none], obj.get("newobjectsSerialized"))
        ml_ranking_model_version = from_none(obj.get("mlRankingModelVersion"))
        return Data(suggestions_query, json_query, query_string, is_newobjects_enabled, is_call_button_enabled, ml_ranking_guid, breadcrumbs, top3_is_on, seo_data, offer_count, offers_serialized, aggregated_count, avg_price_informer, call_button_variant, search_uuid, newobjects_serialized, ml_ranking_model_version)

    def to_dict(self) -> dict:
        result: dict = {}
        result["suggestionsQuery"] = from_none(self.suggestions_query)
        result["jsonQuery"] = from_union([lambda x: to_class(JSONQuery, x), from_none], self.json_query)
        result["queryString"] = from_union([from_str, from_none], self.query_string)
        result["isNewobjectsEnabled"] = from_union([from_bool, from_none], self.is_newobjects_enabled)
        result["isCallButtonEnabled"] = from_union([from_bool, from_none], self.is_call_button_enabled)
        result["mlRankingGuid"] = from_none(self.ml_ranking_guid)
        result["breadcrumbs"] = from_union([lambda x: from_list(lambda x: to_class(Breadcrumb, x), x), from_none], self.breadcrumbs)
        result["top3IsOn"] = from_union([from_bool, from_none], self.top3_is_on)
        result["seoData"] = from_union([lambda x: to_class(SEOData, x), from_none], self.seo_data)
        result["offerCount"] = from_union([from_int, from_none], self.offer_count)
        result["offersSerialized"] = from_union([lambda x: from_list(lambda x: to_class(OffersSerialized, x), x), from_none], self.offers_serialized)
        result["aggregatedCount"] = from_union([from_int, from_none], self.aggregated_count)
        result["avgPriceInformer"] = from_none(self.avg_price_informer)
        result["callButtonVariant"] = from_union([from_int, from_none], self.call_button_variant)
        result["searchUuid"] = from_none(self.search_uuid)
        result["newobjectsSerialized"] = from_union([lambda x: from_list(lambda x: x, x), from_none], self.newobjects_serialized)
        result["mlRankingModelVersion"] = from_none(self.ml_ranking_model_version)
        return result


@dataclass
class Cian:
    status: Optional[str]
    data: Optional[Data]

    @staticmethod
    def from_dict(obj: Any) -> 'Cian':
        assert isinstance(obj, dict)
        status = from_union([from_str, from_none], obj.get("status"))
        data = from_union([Data.from_dict, from_none], obj.get("data"))
        return Cian(status, data)

    def to_dict(self) -> dict:
        result: dict = {}
        result["status"] = from_union([from_str, from_none], self.status)
        result["data"] = from_union([lambda x: to_class(Data, x), from_none], self.data)
        return result


def cian_from_dict(s: Any) -> Cian:
    return Cian.from_dict(s)


def cian_to_dict(x: Cian) -> Any:
    return to_class(Cian, x)


In [10]:
import json
result = cian_from_dict(json.loads(response.json().data.offersSerialized))

TypeError: the JSON object must be str, bytes or bytearray, not dict